In [1]:
%run -i ../header.py
from pylatex import Tabular, MultiColumn, NoEscape, LongTable

In [2]:
data_pack = {
      "b": ["b-trans.deng2017-"+num for num in ("III", "IV", "V", "VI")]
    , "c": ["c-trans.deng2016-"+num for num in ("V", "VI", "VII", "VIII", "IX")]
}

# Export

In [3]:
def getPDGTrans(name):
    # return widths in [KeV]
    pdg_trans = pd.read_csv("../data/{}-trans.PDG.csv".format(name), header=0, index_col=[0,1], comment="#")["frac_width"]
    pdg_trans_err = pd.read_csv("../data/{}-trans.PDG.err.csv".format(name), header=0, index_col=[0,1], comment="#")["frac_width"]
    
    pdg_tot = pd.read_csv("../data/{}-widths.csv".format(name), header=0, index_col=0, comment="#")["PDG"]
    pdg_tot_spec = readSpec("../data/{}-widths.csv".format(name))
    pdg_tot_err = pd.read_csv("../data/{}-widths.err.csv".format(name), header=0, index_col=0, comment="#")["PDG"].abs()
    
    pdg_err = pdg_trans_err.copy()
    
    for instate in pdg_trans.index.levels[0]:
        pdg_err.loc[(instate, slice(None))] = (  pdg_trans_err[(instate, slice(None))].values*pdg_tot[instate]\
                                               + pdg_trans[(instate, slice(None))].values*pdg_tot_err[instate]\
                                              )*dimTrans(pdg_tot_spec["PDG"]["tags"][0], "KeV") # pdg[MeV] -> [KeV]
        pdg_trans.loc[(instate, slice(None))] = pdg_trans[(instate, slice(None))].values*pdg_tot[instate]*dimTrans(pdg_tot_spec["PDG"]["tags"][0], "KeV") # pdg[MeV] -> [KeV]
    
    pdg_trans.index = pd.Index([tuple("${}$".format(code2name(e)) for e in pair) for pair in pdg_trans.index.values], name=["In", "Out"])
    pdg_trans.rename("PDG", inplace=True)
    
    pdg_err.index = pd.Index([tuple("${}$".format(code2name(e)) for e in pair) for pair in pdg_trans_err.index.values], name=["In", "Out"])
    pdg_err = pdg_err.reindex(pdg_trans.index)
    
    return pdg_trans, pdg_err

In [4]:
def getDengTrans(prefix, cfgname):
    # return trans widths of Deng in KeV
    prefixToCol = {"scr": "SP", "lin": "LP"}
    
    col = prefixToCol[prefix]
    spec = readSpec("../data/{}.csv".format(cfgname))
    deng = pd.read_csv("../data/{}.csv".format(cfgname), header=0, index_col=[0,1], comment="#")
    for k, v in spec.items():
        deng[k] *= dimTrans(v["tags"][0], "KeV")
    deng.rename(columns={col: "Deng"}, inplace=True)
    deng.index = pd.Index([tuple("${}$".format(code2name(e)) for e in pair) for pair in deng.index.values], name=["In", "Out"])
    return deng

In [5]:
def getRs(particle, prefix, cfgname, suffix=""):
    # return my widths in KeV
    data = pd.read_csv("output/{}.{}/widths{}".format(prefix, cfgname, suffix), header=0, index_col = [0, 1], comment="#")
    data *= dimTrans("GeV", "KeV")
    data = data.loc[(data != 0).any(axis=1), :]
    data["in0"] /= data["width"]
    data["in2"] /= data["width"]
    data["out0"] /= data["width"]
    data["out2"] /= data["width"]
    data.index.names = ["In", "Out"]
    
    data_sub = data.query("In == '{0}'".format(particle))\
        .rename(columns={"out0": "$r^{(0)}$", "out2": "$r^{(2)}$"})\
        .drop(columns=["in0", "in2"])
    data_above = data.query("Out == '{0}'".format(particle))\
        .rename(columns={"in0": "$r^{(0)}$", "in2": "$r^{(2)}$"})\
        .drop(columns=["out0", "out2"])
    
    data = pd.concat([data_sub, data_above])
    data.columns = ["$\Gamma$", "$r^{(0)}$", "$r^{(2)}$"]
    
    data.index = pd.Index([tuple("${}$".format(code2name(e)) for e in pair) for pair in data.index.values], name=["In", "Out"])
    return data

In [6]:
def reportRs(pname, prefix, cfgname, suffixes=["", "E1"]): # allowed suffixes: "", "E1", "ELW"
    stack = []
    deng_stack = []
    parts = [getRs(pname, prefix, cfgname, suffix)\
                                .add_prefix("" if suffix == "" else suffix+"-")\
                            for suffix in suffixes]
    parts = [part for part in parts if not part.empty]
    if len(parts) > 0:
        stack.append(pd.concat(parts, axis=1))
    
    for dataf in data_pack[cfgname.split("-")[0]]:
        deng_stack.append(getDengTrans(prefix, dataf)["Deng"])
    
    stack = pd.concat(stack, sort=False)
    deng_stack = pd.concat(deng_stack, axis=0, sort=False).reindex(stack.index.values)
    pdg, _ = getPDGTrans(cfgname.split("-")[0])
    pdg = pdg.reindex(stack.index.values)
    
    res = pd.concat([pdg, deng_stack, stack], sort=False, axis=1)
    res = dfsort(res, lambda a,b: cmpMultyStatesByCode(prefix, cfgname
                                                      ,[name2code(i[1:-1]) for i in a]
                                                      ,[name2code(i[1:-1]) for i in b]))
    return res

## Widths

### $\psi(1S)$ and $\Upsilon(1S)$

In [7]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("psi_1S", "scr", "c-scr", suffixes=suff)
    display(res)
    with open("export/width{}.psi_1S.c-scr.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


,,PDG,Deng,$\Gamma$,$r^{(0)}$,$r^{(2)}$,E1-$\Gamma$,E1-$r^{(0)}$,E1-$r^{(2)}$
In,Out,,,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,2.44,2.79,1,0,2.79,1,0
$\chi_{c0}(1P)$,$\psi(1S)$,151,179,142,1,0,176,1,0
$\chi_{c1}(1P)$,$\psi(1S)$,288,319,310,0.569,0,353,0.5,0
$\chi_{c2}(1P)$,$\psi(1S)$,374,292,510,0.143,0.519,441,0.1,0.6
$\eta_{c}(2S)$,$\psi(1S)$,nan,2.29,2.91,1,0,2.91,1,0
$\chi_{c0}(2P)$,$\psi(1S)$,nan,2.3,1.89,1,0,3.11,1,0
$\chi_{c1}(2P)$,$\psi(1S)$,nan,88,74.4,0.623,0,92.7,0.5,0
$\chi_{c2}(2P)$,$\psi(1S)$,nan,93,237,0.17,0.471,186,0.1,0.6


['E1', 'ELW']


,,PDG,Deng,E1-$\Gamma$,E1-$r^{(0)}$,E1-$r^{(2)}$,ELW-$\Gamma$,ELW-$r^{(0)}$,ELW-$r^{(2)}$
In,Out,,,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,2.44,2.79,1,0,nan,nan,nan
$\chi_{c0}(1P)$,$\psi(1S)$,151,179,176,1,0,183,1,0
$\chi_{c1}(1P)$,$\psi(1S)$,288,319,353,0.5,0,382,0.5,0
$\chi_{c2}(1P)$,$\psi(1S)$,374,292,441,0.1,0.6,490,0.1,0.6
$\eta_{c}(2S)$,$\psi(1S)$,nan,2.29,2.91,1,0,nan,nan,nan
$\chi_{c0}(2P)$,$\psi(1S)$,nan,2.3,3.11,1,0,3.3,1,0
$\chi_{c1}(2P)$,$\psi(1S)$,nan,88,92.7,0.5,0,51.9,0.5,0
$\chi_{c2}(2P)$,$\psi(1S)$,nan,93,186,0.1,0.6,132,0.1,0.6


In [8]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("psi_1S", "lin", "c-lin", suffixes=suff)
    display(res)
    with open("export/width{}.psi_1S.c-lin.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


,,PDG,Deng,$\Gamma$,$r^{(0)}$,$r^{(2)}$,E1-$\Gamma$,E1-$r^{(0)}$,E1-$r^{(2)}$
In,Out,,,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,2.39,2.53,1,0,2.53,1,0
$\chi_{c0}(1P)$,$\psi(1S)$,151,172,138,1,0,170,1,0
$\chi_{c1}(1P)$,$\psi(1S)$,288,306,301,0.566,0,341,0.5,0
$\chi_{c2}(1P)$,$\psi(1S)$,374,284,490,0.141,0.523,427,0.1,0.6
$\eta_{c}(2S)$,$\psi(1S)$,nan,2.64,3.04,1,0,3.04,1,0
$\chi_{c0}(2P)$,$\psi(1S)$,nan,6.1,2.66,1,0,4.26,1,0
$\chi_{c1}(2P)$,$\psi(1S)$,nan,81,69.1,0.616,0,85.2,0.5,0
$\chi_{c2}(2P)$,$\psi(1S)$,nan,93,231,0.166,0.477,184,0.1,0.6


['E1', 'ELW']


,,PDG,Deng,E1-$\Gamma$,E1-$r^{(0)}$,E1-$r^{(2)}$,ELW-$\Gamma$,ELW-$r^{(0)}$,ELW-$r^{(2)}$
In,Out,,,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,2.39,2.53,1,0,nan,nan,nan
$\chi_{c0}(1P)$,$\psi(1S)$,151,172,170,1,0,177,1,0
$\chi_{c1}(1P)$,$\psi(1S)$,288,306,341,0.5,0,367,0.5,0
$\chi_{c2}(1P)$,$\psi(1S)$,374,284,427,0.1,0.6,471,0.1,0.6
$\eta_{c}(2S)$,$\psi(1S)$,nan,2.64,3.04,1,0,nan,nan,nan
$\chi_{c0}(2P)$,$\psi(1S)$,nan,6.1,4.26,1,0,1.66,1,0
$\chi_{c1}(2P)$,$\psi(1S)$,nan,81,85.2,0.5,0,46.4,0.5,0
$\chi_{c2}(2P)$,$\psi(1S)$,nan,93,184,0.1,0.6,129,0.1,0.6


In [9]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("yps_1S", "scr", "b-scr", suffixes=suff)
    display(res)
    with open("export/width{}.yps_1S.b-scr.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


,,PDG,Deng,$\Gamma$,$r^{(0)}$,$r^{(2)}$,E1-$\Gamma$,E1-$r^{(0)}$,E1-$r^{(2)}$
In,Out,,,,,,,,
$\Upsilon(1S)$,$\eta_{b}(1S)$,nan,0.01,0.0107,1,0,0.0107,1,0
$\chi_{b0}(1P)$,$\Upsilon(1S)$,nan,27.5,24.2,1,0,26.3,1,0
$\chi_{b1}(1P)$,$\Upsilon(1S)$,nan,31.9,30.2,0.522,0,31.6,0.5,0
$\chi_{b2}(1P)$,$\Upsilon(1S)$,nan,31.8,36.1,0.114,0.573,34.5,0.1,0.6
$\chi_{b0}(2P)$,$\Upsilon(1S)$,nan,5.54,4.44,1,0,5.19,1,0
$\chi_{b1}(2P)$,$\Upsilon(1S)$,nan,10.8,10.7,0.54,0,11.6,0.5,0
$\chi_{b2}(2P)$,$\Upsilon(1S)$,nan,12.5,17,0.124,0.554,15.7,0.1,0.6
$\chi_{b0}(3P)$,$\Upsilon(1S)$,nan,1.87,1.37,1,0,1.69,1,0
$\chi_{b1}(3P)$,$\Upsilon(1S)$,nan,6.41,5.56,0.552,0,6.15,0.5,0


['E1', 'ELW']


,,PDG,Deng,E1-$\Gamma$,E1-$r^{(0)}$,E1-$r^{(2)}$,ELW-$\Gamma$,ELW-$r^{(0)}$,ELW-$r^{(2)}$
In,Out,,,,,,,,
$\Upsilon(1S)$,$\eta_{b}(1S)$,nan,0.01,0.0107,1,0,nan,nan,nan
$\chi_{b0}(1P)$,$\Upsilon(1S)$,nan,27.5,26.3,1,0,26.9,1,0
$\chi_{b1}(1P)$,$\Upsilon(1S)$,nan,31.9,31.6,0.5,0,32.5,0.5,0
$\chi_{b2}(1P)$,$\Upsilon(1S)$,nan,31.8,34.5,0.1,0.6,35.7,0.1,0.6
$\chi_{b0}(2P)$,$\Upsilon(1S)$,nan,5.54,5.19,1,0,3.97,1,0
$\chi_{b1}(2P)$,$\Upsilon(1S)$,nan,10.8,11.6,0.5,0,10,0.5,0
$\chi_{b2}(2P)$,$\Upsilon(1S)$,nan,12.5,15.7,0.1,0.6,14.1,0.1,0.6
$\chi_{b0}(3P)$,$\Upsilon(1S)$,nan,1.87,1.69,1,0,1.01,1,0
$\chi_{b1}(3P)$,$\Upsilon(1S)$,nan,6.41,6.15,0.5,0,4.83,0.5,0


### $\psi(2S)$ and $\Upsilon(2S)$

In [10]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("psi_2S", "scr", "c-scr", suffixes=suff)
    display(res)
    with open("export/width{}.psi_2S.c-scr.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


PDG  Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In              Out                                                           
$\psi(2S)$      $\eta_{c}(1S)$      1   7.8      10.6          1          0   
                $\chi_{c0}(1P)$  28.8    22      36.6          1          0   
                $\chi_{c1}(1P)$  28.7    45        56      0.471          0   
                $\chi_{c2}(1P)$    28    46      44.5     0.0868      0.627   
                $\eta_{c}(2S)$  0.206  0.19     0.233          1          0   
$\chi_{c0}(2P)$ $\psi(2S)$        nan    99       108          1          0   
$\chi_{c1}(2P)$ $\psi(2S)$        nan   155       102      0.532          0   
$\chi_{c2}(2P)$ $\psi(2S)$        nan   150       200      0.124      0.553   
$\chi_{c0}(3P)$ $\psi(2S)$        nan   9.1      6.62          1          0   
$\chi_{c1}(3P)$ $\psi(2S)$        nan    74      68.1      0.585          0   
$\chi_{c2}(3P)$ $\psi(2S)$        nan    76       136      0.149      0.508   

                                 E1-$\Gamma$  E1-$r^{(0)}$  E1-$r^{(2)}$  
In              Out                                                       
$\psi(2S)$      $\eta_{c}(1S)$          10.6             1             0  
                $\chi_{c0}(1P)$         30.9             1             0  
                $\chi_{c1}(1P)$         52.8           0.5             0  
                $\chi_{c2}(1P)$         46.5           0.1           0.6  
                $\eta_{c}(2S)$         0.233             1             0  
$\chi_{c0}(2P)$ $\psi(2S)$               122             1             0  
$\chi_{c1}(2P)$ $\psi(2S)$               109           0.5             0  
$\chi_{c2}(2P)$ $\psi(2S)$               185           0.1           0.6  
$\chi_{c0}(3P)$ $\psi(2S)$              9.07             1             0  
$\chi_{c1}(3P)$ $\psi(2S)$              79.7           0.5             0  
$\chi_{c2}(3P)$ $\psi(2S)$               115           0.1           0.6

['E1', 'ELW']


PDG  Deng  E1-$\Gamma$  E1-$r^{(0)}$  \
In              Out                                                      
$\psi(2S)$      $\eta_{c}(1S)$      1   7.8         10.6             1   
                $\chi_{c0}(1P)$  28.8    22         30.9             1   
                $\chi_{c1}(1P)$  28.7    45         52.8           0.5   
                $\chi_{c2}(1P)$    28    46         46.5           0.1   
                $\eta_{c}(2S)$  0.206  0.19        0.233             1   
$\chi_{c0}(2P)$ $\psi(2S)$        nan    99          122             1   
$\chi_{c1}(2P)$ $\psi(2S)$        nan   155          109           0.5   
$\chi_{c2}(2P)$ $\psi(2S)$        nan   150          185           0.1   
$\chi_{c0}(3P)$ $\psi(2S)$        nan   9.1         9.07             1   
$\chi_{c1}(3P)$ $\psi(2S)$        nan    74         79.7           0.5   
$\chi_{c2}(3P)$ $\psi(2S)$        nan    76          115           0.1   

                                 E1-$r^{(2)}$  ELW-$\Gamma$  ELW-$r^{(0)}$  \
In              Out                                                          
$\psi(2S)$      $\eta_{c}(1S)$              0           nan            nan   
                $\chi_{c0}(1P)$             0          33.5              1   
                $\chi_{c1}(1P)$             0          54.7            0.5   
                $\chi_{c2}(1P)$           0.6          47.5            0.1   
                $\eta_{c}(2S)$              0           nan            nan   
$\chi_{c0}(2P)$ $\psi(2S)$                  0           128              1   
$\chi_{c1}(2P)$ $\psi(2S)$                  0           116            0.5   
$\chi_{c2}(2P)$ $\psi(2S)$                0.6           207            0.1   
$\chi_{c0}(3P)$ $\psi(2S)$                  0        0.0396              1   
$\chi_{c1}(3P)$ $\psi(2S)$                  0          40.1            0.5   
$\chi_{c2}(3P)$ $\psi(2S)$                0.6          75.1            0.1   

                                 ELW-$r^{(2)}$  
In              Out                             
$\psi(2S)$      $\eta_{c}(1S)$             nan  
                $\chi_{c0}(1P)$              0  
                $\chi_{c1}(1P)$              0  
                $\chi_{c2}(1P)$            0.6  
                $\eta_{c}(2S)$             nan  
$\chi_{c0}(2P)$ $\psi(2S)$                   0  
$\chi_{c1}(2P)$ $\psi(2S)$                   0  
$\chi_{c2}(2P)$ $\psi(2S)$                 0.6  
$\chi_{c0}(3P)$ $\psi(2S)$                   0  
$\chi_{c1}(3P)$ $\psi(2S)$                   0  
$\chi_{c2}(3P)$ $\psi(2S)$                 0.6

In [11]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("psi_2S", "lin", "c-lin", suffixes=suff)
    display(res)
    with open("export/width{}.psi_2S.c-lin.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


PDG  Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In              Out                                                           
$\psi(2S)$      $\eta_{c}(1S)$      1  8.08      9.98          1          0   
                $\chi_{c0}(1P)$  28.8    22      37.2          1          0   
                $\chi_{c1}(1P)$  28.7    42      52.2      0.473          0   
                $\chi_{c2}(1P)$    28    38      41.8     0.0874      0.626   
                $\eta_{c}(2S)$  0.206  0.19     0.211          1          0   
$\chi_{c0}(2P)$ $\psi(2S)$        nan   121      94.4          1          0   
$\chi_{c1}(2P)$ $\psi(2S)$        nan   139        91      0.531          0   
$\chi_{c2}(2P)$ $\psi(2S)$        nan   135       177      0.123      0.555   
$\chi_{c0}(3P)$ $\psi(2S)$        nan    17      10.4          1          0   
$\chi_{c1}(3P)$ $\psi(2S)$        nan    94      86.6      0.595          0   
$\chi_{c2}(3P)$ $\psi(2S)$        nan    97       192      0.154      0.498   

                                 E1-$\Gamma$  E1-$r^{(0)}$  E1-$r^{(2)}$  
In              Out                                                       
$\psi(2S)$      $\eta_{c}(1S)$          9.98             1             0  
                $\chi_{c0}(1P)$         31.6             1             0  
                $\chi_{c1}(1P)$         49.3           0.5             0  
                $\chi_{c2}(1P)$         43.5           0.1           0.6  
                $\eta_{c}(2S)$         0.211             1             0  
$\chi_{c0}(2P)$ $\psi(2S)$               106             1             0  
$\chi_{c1}(2P)$ $\psi(2S)$              96.6           0.5             0  
$\chi_{c2}(2P)$ $\psi(2S)$               164           0.1           0.6  
$\chi_{c0}(3P)$ $\psi(2S)$              14.8             1             0  
$\chi_{c1}(3P)$ $\psi(2S)$               103           0.5             0  
$\chi_{c2}(3P)$ $\psi(2S)$               160           0.1           0.6

['E1', 'ELW']


PDG  Deng  E1-$\Gamma$  E1-$r^{(0)}$  \
In              Out                                                      
$\psi(2S)$      $\eta_{c}(1S)$      1  8.08         9.98             1   
                $\chi_{c0}(1P)$  28.8    22         31.6             1   
                $\chi_{c1}(1P)$  28.7    42         49.3           0.5   
                $\chi_{c2}(1P)$    28    38         43.5           0.1   
                $\eta_{c}(2S)$  0.206  0.19        0.211             1   
$\chi_{c0}(2P)$ $\psi(2S)$        nan   121          106             1   
$\chi_{c1}(2P)$ $\psi(2S)$        nan   139         96.6           0.5   
$\chi_{c2}(2P)$ $\psi(2S)$        nan   135          164           0.1   
$\chi_{c0}(3P)$ $\psi(2S)$        nan    17         14.8             1   
$\chi_{c1}(3P)$ $\psi(2S)$        nan    94          103           0.5   
$\chi_{c2}(3P)$ $\psi(2S)$        nan    97          160           0.1   

                                 E1-$r^{(2)}$  ELW-$\Gamma$  ELW-$r^{(0)}$  \
In              Out                                                          
$\psi(2S)$      $\eta_{c}(1S)$              0           nan            nan   
                $\chi_{c0}(1P)$             0          34.1              1   
                $\chi_{c1}(1P)$             0            51            0.5   
                $\chi_{c2}(1P)$           0.6          44.4            0.1   
                $\eta_{c}(2S)$              0           nan            nan   
$\chi_{c0}(2P)$ $\psi(2S)$                  0           111              1   
$\chi_{c1}(2P)$ $\psi(2S)$                  0           102            0.5   
$\chi_{c2}(2P)$ $\psi(2S)$                0.6           181            0.1   
$\chi_{c0}(3P)$ $\psi(2S)$                  0         0.466              1   
$\chi_{c1}(3P)$ $\psi(2S)$                  0          37.5            0.5   
$\chi_{c2}(3P)$ $\psi(2S)$                0.6          86.7            0.1   

                                 ELW-$r^{(2)}$  
In              Out                             
$\psi(2S)$      $\eta_{c}(1S)$             nan  
                $\chi_{c0}(1P)$              0  
                $\chi_{c1}(1P)$              0  
                $\chi_{c2}(1P)$            0.6  
                $\eta_{c}(2S)$             nan  
$\chi_{c0}(2P)$ $\psi(2S)$                   0  
$\chi_{c1}(2P)$ $\psi(2S)$                   0  
$\chi_{c2}(2P)$ $\psi(2S)$                 0.6  
$\chi_{c0}(3P)$ $\psi(2S)$                   0  
$\chi_{c1}(3P)$ $\psi(2S)$                   0  
$\chi_{c2}(3P)$ $\psi(2S)$                 0.6

In [12]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("yps_2S", "scr", "b-scr", suffixes=suff)
    display(res)
    with open("export/width{}.yps_2S.b-scr.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


PDG  Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In              Out                                                            
$\Upsilon(2S)$  $\eta_{b}(1S)$  0.0125 0.066    0.0737          1          0   
                $\chi_{b0}(1P)$   1.22  1.09      1.19          1          0   
                $\chi_{b1}(1P)$   2.21  2.17      2.28      0.493          0   
                $\chi_{b2}(1P)$   2.29  2.62      2.58     0.0965      0.607   
$\chi_{b0}(2P)$ $\Upsilon(2S)$     nan  14.4      13.2          1          0   
$\chi_{b1}(2P)$ $\Upsilon(2S)$     nan  15.3      15.3      0.512          0   
$\chi_{b2}(2P)$ $\Upsilon(2S)$     nan  15.3      16.7      0.108      0.585   
$\chi_{b0}(3P)$ $\Upsilon(2S)$     nan  2.55      2.18          1          0   
$\chi_{b1}(3P)$ $\Upsilon(2S)$     nan  5.63      5.04      0.525          0   
$\chi_{b2}(3P)$ $\Upsilon(2S)$     nan  6.72      7.51      0.115       0.57   

                                 E1-$\Gamma$  E1-$r^{(0)}$  E1-$r^{(2)}$  
In              Out                                                       
$\Upsilon(2S)$  $\eta_{b}(1S)$        0.0737             1             0  
                $\chi_{b0}(1P)$         1.15             1             0  
                $\chi_{b1}(1P)$         2.25           0.5             0  
                $\chi_{b2}(1P)$         2.61           0.1           0.6  
$\chi_{b0}(2P)$ $\Upsilon(2S)$          13.8             1             0  
$\chi_{b1}(2P)$ $\Upsilon(2S)$          15.6           0.5             0  
$\chi_{b2}(2P)$ $\Upsilon(2S)$          16.3           0.1           0.6  
$\chi_{b0}(3P)$ $\Upsilon(2S)$           2.4             1             0  
$\chi_{b1}(3P)$ $\Upsilon(2S)$          5.29           0.5             0  
$\chi_{b2}(3P)$ $\Upsilon(2S)$          7.13           0.1           0.6

['E1', 'ELW']


PDG  Deng  E1-$\Gamma$  E1-$r^{(0)}$  \
In              Out                                                       
$\Upsilon(2S)$  $\eta_{b}(1S)$  0.0125 0.066       0.0737             1   
                $\chi_{b0}(1P)$   1.22  1.09         1.15             1   
                $\chi_{b1}(1P)$   2.21  2.17         2.25           0.5   
                $\chi_{b2}(1P)$   2.29  2.62         2.61           0.1   
$\chi_{b0}(2P)$ $\Upsilon(2S)$     nan  14.4         13.8             1   
$\chi_{b1}(2P)$ $\Upsilon(2S)$     nan  15.3         15.6           0.5   
$\chi_{b2}(2P)$ $\Upsilon(2S)$     nan  15.3         16.3           0.1   
$\chi_{b0}(3P)$ $\Upsilon(2S)$     nan  2.55          2.4             1   
$\chi_{b1}(3P)$ $\Upsilon(2S)$     nan  5.63         5.29           0.5   
$\chi_{b2}(3P)$ $\Upsilon(2S)$     nan  6.72         7.13           0.1   

                                 E1-$r^{(2)}$  ELW-$\Gamma$  ELW-$r^{(0)}$  \
In              Out                                                          
$\Upsilon(2S)$  $\eta_{b}(1S)$              0           nan            nan   
                $\chi_{b0}(1P)$             0          1.16              1   
                $\chi_{b1}(1P)$             0          2.26            0.5   
                $\chi_{b2}(1P)$           0.6          2.62            0.1   
$\chi_{b0}(2P)$ $\Upsilon(2S)$              0          14.2              1   
$\chi_{b1}(2P)$ $\Upsilon(2S)$              0          16.2            0.5   
$\chi_{b2}(2P)$ $\Upsilon(2S)$            0.6          16.9            0.1   
$\chi_{b0}(3P)$ $\Upsilon(2S)$              0          1.62              1   
$\chi_{b1}(3P)$ $\Upsilon(2S)$              0          4.23            0.5   
$\chi_{b2}(3P)$ $\Upsilon(2S)$            0.6          5.98            0.1   

                                 ELW-$r^{(2)}$  
In              Out                             
$\Upsilon(2S)$  $\eta_{b}(1S)$             nan  
                $\chi_{b0}(1P)$              0  
                $\chi_{b1}(1P)$              0  
                $\chi_{b2}(1P)$            0.6  
$\chi_{b0}(2P)$ $\Upsilon(2S)$               0  
$\chi_{b1}(2P)$ $\Upsilon(2S)$               0  
$\chi_{b2}(2P)$ $\Upsilon(2S)$             0.6  
$\chi_{b0}(3P)$ $\Upsilon(2S)$               0  
$\chi_{b1}(3P)$ $\Upsilon(2S)$               0  
$\chi_{b2}(3P)$ $\Upsilon(2S)$             0.6

### $\psi(3S)$ and $\Upsilon(3S)$

In [13]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("psi_3S", "scr", "c-scr", suffixes=suff)
    display(res)
    with open("export/width{}.psi_3S.c-scr.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


PDG  Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In         Out                                                          
$\psi(3S)$ $\eta_{c}(1S)$   nan  6.76      9.95          1          0   
           $\chi_{c0}(1P)$  nan   6.7      20.6          1          0   
           $\chi_{c1}(1P)$  nan   6.7        12      0.425          0   
           $\chi_{c2}(1P)$  nan   2.5      2.18     0.0563      0.694   
           $\eta_{c}(2S)$   nan  1.78       2.2          1          0   
           $\chi_{c0}(2P)$  nan    27      30.6          1          0   
           $\chi_{c1}(2P)$  nan    67       132      0.472          0   
           $\chi_{c2}(2P)$  nan    82      78.4     0.0885      0.623   
           $\eta_{c}(3S)$   nan 0.088     0.087          1          0   

                            E1-$\Gamma$  E1-$r^{(0)}$  E1-$r^{(2)}$  
In         Out                                                       
$\psi(3S)$ $\eta_{c}(1S)$          9.95             1             0  
           $\chi_{c0}(1P)$         14.6             1             0  
           $\chi_{c1}(1P)$         10.2           0.5             0  
           $\chi_{c2}(1P)$         2.52           0.1           0.6  
           $\eta_{c}(2S)$           2.2             1             0  
           $\chi_{c0}(2P)$         27.2             1             0  
           $\chi_{c1}(2P)$          125           0.5             0  
           $\chi_{c2}(2P)$         81.4           0.1           0.6  
           $\eta_{c}(3S)$         0.087             1             0

['E1', 'ELW']


PDG  Deng  E1-$\Gamma$  E1-$r^{(0)}$  \
In         Out                                                     
$\psi(3S)$ $\eta_{c}(1S)$   nan  6.76         9.95             1   
           $\chi_{c0}(1P)$  nan   6.7         14.6             1   
           $\chi_{c1}(1P)$  nan   6.7         10.2           0.5   
           $\chi_{c2}(1P)$  nan   2.5         2.52           0.1   
           $\eta_{c}(2S)$   nan  1.78          2.2             1   
           $\chi_{c0}(2P)$  nan    27         27.2             1   
           $\chi_{c1}(2P)$  nan    67          125           0.5   
           $\chi_{c2}(2P)$  nan    82         81.4           0.1   
           $\eta_{c}(3S)$   nan 0.088        0.087             1   

                            E1-$r^{(2)}$  ELW-$\Gamma$  ELW-$r^{(0)}$  \
In         Out                                                          
$\psi(3S)$ $\eta_{c}(1S)$              0           nan            nan   
           $\chi_{c0}(1P)$             0          8.15              1   
           $\chi_{c1}(1P)$             0         0.753            0.5   
           $\chi_{c2}(1P)$           0.6             2            0.1   
           $\eta_{c}(2S)$              0           nan            nan   
           $\chi_{c0}(2P)$             0          29.8              1   
           $\chi_{c1}(2P)$             0           135            0.5   
           $\chi_{c2}(2P)$           0.6          84.4            0.1   
           $\eta_{c}(3S)$              0           nan            nan   

                            ELW-$r^{(2)}$  
In         Out                             
$\psi(3S)$ $\eta_{c}(1S)$             nan  
           $\chi_{c0}(1P)$              0  
           $\chi_{c1}(1P)$              0  
           $\chi_{c2}(1P)$            0.6  
           $\eta_{c}(2S)$             nan  
           $\chi_{c0}(2P)$              0  
           $\chi_{c1}(2P)$              0  
           $\chi_{c2}(2P)$            0.6  
           $\eta_{c}(3S)$             nan

In [14]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("psi_3S", "lin", "c-lin", suffixes=suff)
    display(res)
    with open("export/width{}.psi_3S.c-lin.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


PDG  Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In         Out                                                          
$\psi(3S)$ $\eta_{c}(1S)$   nan  6.66      9.06          1          0   
           $\chi_{c0}(1P)$  nan   5.9      17.3          1          0   
           $\chi_{c1}(1P)$  nan     4         7      0.428          0   
           $\chi_{c2}(1P)$  nan  0.25     0.148     0.0583      0.689   
           $\eta_{c}(2S)$   nan  1.65      1.85          1          0   
           $\chi_{c0}(2P)$  nan    19      27.2          1          0   
           $\chi_{c1}(2P)$  nan    55       107      0.474          0   
           $\chi_{c2}(2P)$  nan    67      63.7     0.0891      0.622   

                            E1-$\Gamma$  E1-$r^{(0)}$  E1-$r^{(2)}$  
In         Out                                                       
$\psi(3S)$ $\eta_{c}(1S)$          9.06             1             0  
           $\chi_{c0}(1P)$         12.4             1             0  
           $\chi_{c1}(1P)$         5.99           0.5             0  
           $\chi_{c2}(1P)$         0.17           0.1           0.6  
           $\eta_{c}(2S)$          1.85             1             0  
           $\chi_{c0}(2P)$         24.3             1             0  
           $\chi_{c1}(2P)$          102           0.5             0  
           $\chi_{c2}(2P)$         66.1           0.1           0.6

['E1', 'ELW']


PDG  Deng  E1-$\Gamma$  E1-$r^{(0)}$  \
In         Out                                                     
$\psi(3S)$ $\eta_{c}(1S)$   nan  6.66         9.06             1   
           $\chi_{c0}(1P)$  nan   5.9         12.4             1   
           $\chi_{c1}(1P)$  nan     4         5.99           0.5   
           $\chi_{c2}(1P)$  nan  0.25         0.17           0.1   
           $\eta_{c}(2S)$   nan  1.65         1.85             1   
           $\chi_{c0}(2P)$  nan    19         24.3             1   
           $\chi_{c1}(2P)$  nan    55          102           0.5   
           $\chi_{c2}(2P)$  nan    67         66.1           0.1   

                            E1-$r^{(2)}$  ELW-$\Gamma$  ELW-$r^{(0)}$  \
In         Out                                                          
$\psi(3S)$ $\eta_{c}(1S)$              0           nan            nan   
           $\chi_{c0}(1P)$             0          6.21              1   
           $\chi_{c1}(1P)$             0        0.0879            0.5   
           $\chi_{c2}(1P)$           0.6          5.56            0.1   
           $\eta_{c}(2S)$              0           nan            nan   
           $\chi_{c0}(2P)$             0          26.3              1   
           $\chi_{c1}(2P)$             0           109            0.5   
           $\chi_{c2}(2P)$           0.6            68            0.1   

                            ELW-$r^{(2)}$  
In         Out                             
$\psi(3S)$ $\eta_{c}(1S)$             nan  
           $\chi_{c0}(1P)$              0  
           $\chi_{c1}(1P)$              0  
           $\chi_{c2}(1P)$            0.6  
           $\eta_{c}(2S)$             nan  
           $\chi_{c0}(2P)$              0  
           $\chi_{c1}(2P)$              0  
           $\chi_{c2}(2P)$            0.6

In [15]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("yps_3S", "scr", "b-scr", suffixes=suff)
    display(res)
    with open("export/width{}.yps_3S.b-scr.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


PDG   Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In              Out                                                             
$\Upsilon(3S)$  $\eta_{b}(1S)$  0.0104  0.071    0.0795          1          0   
                $\chi_{b0}(1P)$ 0.0549  0.097     0.116          1          0   
                $\chi_{b1}(1P)$ 0.0183 0.0005  9.28e-05      0.477          0   
                $\chi_{b2}(1P)$  0.201   0.14     0.153     0.0867      0.627   
                $\eta_{b}(2S)$     nan  0.011    0.0122          1          0   
                $\chi_{b0}(2P)$    1.2   1.21      1.31          1          0   
                $\chi_{b1}(2P)$   2.56   2.61      2.66      0.495          0   
                $\chi_{b2}(2P)$   2.66   3.16      3.18     0.0973      0.605   
$\chi_{b0}(3P)$ $\Upsilon(3S)$     nan   7.95      7.59          1          0   
$\chi_{b1}(3P)$ $\Upsilon(3S)$     nan   10.3      9.38      0.508          0   
$\chi_{b2}(3P)$ $\Upsilon(3S)$     nan   10.8      11.2      0.105      0.589   

                                 E1-$\Gamma$  E1-$r^{(0)}$  E1-$r^{(2)}$  
In              Out                                                       
$\Upsilon(3S)$  $\eta_{b}(1S)$        0.0795             1             0  
                $\chi_{b0}(1P)$        0.105             1             0  
                $\chi_{b1}(1P)$     8.86e-05           0.5             0  
                $\chi_{b2}(1P)$         0.16           0.1           0.6  
                $\eta_{b}(2S)$        0.0122             1             0  
                $\chi_{b0}(2P)$         1.28             1             0  
                $\chi_{b1}(2P)$         2.63           0.5             0  
                $\chi_{b2}(2P)$         3.21           0.1           0.6  
$\chi_{b0}(3P)$ $\Upsilon(3S)$          7.81             1             0  
$\chi_{b1}(3P)$ $\Upsilon(3S)$          9.54           0.5             0  
$\chi_{b2}(3P)$ $\Upsilon(3S)$            11           0.1           0.6

['E1', 'ELW']


PDG   Deng  E1-$\Gamma$  E1-$r^{(0)}$  \
In              Out                                                        
$\Upsilon(3S)$  $\eta_{b}(1S)$  0.0104  0.071       0.0795             1   
                $\chi_{b0}(1P)$ 0.0549  0.097        0.105             1   
                $\chi_{b1}(1P)$ 0.0183 0.0005     8.86e-05           0.5   
                $\chi_{b2}(1P)$  0.201   0.14         0.16           0.1   
                $\eta_{b}(2S)$     nan  0.011       0.0122             1   
                $\chi_{b0}(2P)$    1.2   1.21         1.28             1   
                $\chi_{b1}(2P)$   2.56   2.61         2.63           0.5   
                $\chi_{b2}(2P)$   2.66   3.16         3.21           0.1   
$\chi_{b0}(3P)$ $\Upsilon(3S)$     nan   7.95         7.81             1   
$\chi_{b1}(3P)$ $\Upsilon(3S)$     nan   10.3         9.54           0.5   
$\chi_{b2}(3P)$ $\Upsilon(3S)$     nan   10.8           11           0.1   

                                 E1-$r^{(2)}$  ELW-$\Gamma$  ELW-$r^{(0)}$  \
In              Out                                                          
$\Upsilon(3S)$  $\eta_{b}(1S)$              0           nan            nan   
                $\chi_{b0}(1P)$             0        0.0436              1   
                $\chi_{b1}(1P)$             0        0.0413            0.5   
                $\chi_{b2}(1P)$           0.6         0.455            0.1   
                $\eta_{b}(2S)$              0           nan            nan   
                $\chi_{b0}(2P)$             0           1.3              1   
                $\chi_{b1}(2P)$             0          2.65            0.5   
                $\chi_{b2}(2P)$           0.6          3.23            0.1   
$\chi_{b0}(3P)$ $\Upsilon(3S)$              0          7.98              1   
$\chi_{b1}(3P)$ $\Upsilon(3S)$              0          9.84            0.5   
$\chi_{b2}(3P)$ $\Upsilon(3S)$            0.6          11.5            0.1   

                                 ELW-$r^{(2)}$  
In              Out                             
$\Upsilon(3S)$  $\eta_{b}(1S)$             nan  
                $\chi_{b0}(1P)$              0  
                $\chi_{b1}(1P)$              0  
                $\chi_{b2}(1P)$            0.6  
                $\eta_{b}(2S)$             nan  
                $\chi_{b0}(2P)$              0  
                $\chi_{b1}(2P)$              0  
                $\chi_{b2}(2P)$            0.6  
$\chi_{b0}(3P)$ $\Upsilon(3S)$               0  
$\chi_{b1}(3P)$ $\Upsilon(3S)$               0  
$\chi_{b2}(3P)$ $\Upsilon(3S)$             0.6

### $\psi_1(1D)$ which is $\psi(3770)$

In [16]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("psi_1_1D", "scr", "c-scr", suffixes=suff)
    display(res)
    with open("export/width{}.psi_1_1D.c-scr.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


PDG  Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In              Out                                                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan   nan     0.386          1          0   
                $\chi_{c0}(1P)$  188   261       253          1          0   
                $\chi_{c1}(1P)$ 67.7   135       115      0.538          0   
                $\chi_{c2}(1P)$  nan   8.1      4.73      0.233      0.535   
                $\eta_{c}(2S)$   nan   nan   0.00011          1          0   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan    12      22.5          1          0   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan   9.8      5.67      0.488          0   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan  0.46     0.812      0.121      0.697   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan  0.39     0.448          1          0   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan     2      4.35      0.441          0   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan  0.79       1.5      0.177     0.0495   

                                 E1-$\Gamma$  E1-$r^{(0)}$  E1-$r^{(2)}$  
In              Out                                                       
$\psi_{1}(1D)$  $\eta_{c}(1S)$         0.386             1             0  
                $\chi_{c0}(1P)$          285             1             0  
                $\chi_{c1}(1P)$          138           0.5             0  
                $\chi_{c2}(1P)$         6.29           0.1           0.6  
                $\eta_{c}(2S)$       0.00011             1             0  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$          21.8             1             0  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$          5.32           0.5             0  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$         0.656           0.1           0.6  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$         0.399             1             0  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$          3.38           0.5             0  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$          0.25           0.1           0.6

['E1', 'ELW']


PDG  Deng  E1-$\Gamma$  E1-$r^{(0)}$  \
In              Out                                                     
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan   nan        0.386             1   
                $\chi_{c0}(1P)$  188   261          285             1   
                $\chi_{c1}(1P)$ 67.7   135          138           0.5   
                $\chi_{c2}(1P)$  nan   8.1         6.29           0.1   
                $\eta_{c}(2S)$   nan   nan      0.00011             1   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan    12         21.8             1   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan   9.8         5.32           0.5   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan  0.46        0.656           0.1   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan  0.39        0.399             1   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan     2         3.38           0.5   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan  0.79         0.25           0.1   

                                 E1-$r^{(2)}$  ELW-$\Gamma$  ELW-$r^{(0)}$  \
In              Out                                                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$              0           nan            nan   
                $\chi_{c0}(1P)$             0           318              1   
                $\chi_{c1}(1P)$             0           148            0.5   
                $\chi_{c2}(1P)$           0.6          6.61            0.1   
                $\eta_{c}(2S)$              0           nan            nan   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$              0          22.1              1   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$              0          5.42            0.5   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$            0.6         0.692            0.1   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$              0          9.49              1   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$              0         0.392            0.5   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$            0.6        0.0791            0.1   

                                 ELW-$r^{(2)}$  
In              Out                             
$\psi_{1}(1D)$  $\eta_{c}(1S)$             nan  
                $\chi_{c0}(1P)$              0  
                $\chi_{c1}(1P)$              0  
                $\chi_{c2}(1P)$            0.6  
                $\eta_{c}(2S)$             nan  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$               0  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$               0  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$             0.6  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$               0  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$               0  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$             0.6

In [17]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("psi_1_1D", "lin", "c-lin", suffixes=suff)
    display(res)
    with open("export/width{}.psi_1_1D.c-lin.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


PDG  Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In              Out                                                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan   nan     0.341          1          0   
                $\chi_{c0}(1P)$  188   272       260          1          0   
                $\chi_{c1}(1P)$ 67.7   138       111      0.536          0   
                $\chi_{c2}(1P)$  nan   7.1      4.53      0.221      0.538   
                $\eta_{c}(2S)$   nan   nan  7.92e-05          1          0   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan    20        18          1          0   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan   7.9      4.48      0.488          0   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan  0.36     0.615      0.117      0.691   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan  0.27     0.237          1          0   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan   3.2      7.39      0.433          0   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan   1.5      2.94      0.202     0.0338   

                                 E1-$\Gamma$  E1-$r^{(0)}$  E1-$r^{(2)}$  
In              Out                                                       
$\psi_{1}(1D)$  $\eta_{c}(1S)$         0.341             1             0  
                $\chi_{c0}(1P)$          291             1             0  
                $\chi_{c1}(1P)$          132           0.5             0  
                $\chi_{c2}(1P)$         5.96           0.1           0.6  
                $\eta_{c}(2S)$      7.92e-05             1             0  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$          17.5             1             0  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$          4.22           0.5             0  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$         0.503           0.1           0.6  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$         0.208             1             0  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$          5.55           0.5             0  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$          0.39           0.1           0.6

['E1', 'ELW']


PDG  Deng  E1-$\Gamma$  E1-$r^{(0)}$  \
In              Out                                                     
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan   nan        0.341             1   
                $\chi_{c0}(1P)$  188   272          291             1   
                $\chi_{c1}(1P)$ 67.7   138          132           0.5   
                $\chi_{c2}(1P)$  nan   7.1         5.96           0.1   
                $\eta_{c}(2S)$   nan   nan     7.92e-05             1   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan    20         17.5             1   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan   7.9         4.22           0.5   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan  0.36        0.503           0.1   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan  0.27        0.208             1   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan   3.2         5.55           0.5   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan   1.5         0.39           0.1   

                                 E1-$r^{(2)}$  ELW-$\Gamma$  ELW-$r^{(0)}$  \
In              Out                                                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$              0           nan            nan   
                $\chi_{c0}(1P)$             0           321              1   
                $\chi_{c1}(1P)$             0           140            0.5   
                $\chi_{c2}(1P)$           0.6          6.24            0.1   
                $\eta_{c}(2S)$              0           nan            nan   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$              0          17.7              1   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$              0          4.29            0.5   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$            0.6         0.527            0.1   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$              0          9.27              1   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$              0         0.345            0.5   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$            0.6        0.0916            0.1   

                                 ELW-$r^{(2)}$  
In              Out                             
$\psi_{1}(1D)$  $\eta_{c}(1S)$             nan  
                $\chi_{c0}(1P)$              0  
                $\chi_{c1}(1P)$              0  
                $\chi_{c2}(1P)$            0.6  
                $\eta_{c}(2S)$             nan  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$               0  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$               0  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$             0.6  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$               0  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$               0  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$             0.6

## Sumrules for the thesis

In [18]:
def getMasses(prefix, cfgname):
    # return masses in MeV
    with open("../quarkEigen/output/{}.{}/config".format(prefix, cfgname), "r") as f:
        mass_cfg = json.load(f, object_pairs_hook=OrderedDict)
    with open("../quarkEigen/output/{}.{}/exclude".format(prefix, cfgname), "r") as f:
        mass_excl = f.read().split("\n")
    mass_data = OrderedDict()
    for particle, pdata in mass_cfg.items():
        if particle not in mass_excl:
            continue
        energ_data = pd.read_csv("../quarkEigen/output/{}.{}/data/{}/minE.dat".format(prefix, cfgname, particle))\
                     .iloc[-1, 1]
        energ = pdata["eq"]["env"]["mC"]*2 + energ_data
        mass_data.update({particle: energ})
    mass_data = pd.DataFrame.from_dict(mass_data, orient="index")
    mass_data.columns = ["E"]
    mass_data["E"] *= dimTrans("GeV", "MeV")
    pdg_spec = readSpec("../data/{}-spectrum.csv".format(cfgname.split("-")[0]))["PDG"]
    pdg = pd.read_csv("../data/{}-spectrum.csv".format(cfgname.split("-")[0]), header=0, index_col=0, comment="#")["PDG"]\
        *dimTrans(pdg_spec["tags"][0], "MeV")
    pdg = pdg.loc[pdg.index.intersection(mass_data.index).values]
    mass_data.loc[~pd.isna(pdg), "E"] = pdg.loc[~pd.isna(pdg)]
    mass_data.index = pd.Index(["${}$".format(code2name(k)) for k in mass_data.index.values], name="States")
    return mass_data

In [19]:
def getPprops(prefix, cfgname):
    with open("../quarkEigen/output/{}.{}/config".format(prefix, cfgname), "r") as f:
        pprops = json.load(f, object_pairs_hook=OrderedDict)
    props = pd.DataFrame(OrderedDict([(particle, values["eq"]) for particle, values in pprops.items()])).T
    props.drop(columns=["env", "E"], inplace=True)
    props.index = pd.Index([r"${}$".format(code2name(particle)) for particle in pprops.keys()], name="States")
    return props

In [20]:
def getSumrule(pname, prefix, cfgname, suffix=""):
    msuffix = "" if suffix == "" else suffix+"-" 
    pretty_pname = "${}$".format(code2name(pname))
    
    widths = reportRs(pname, prefix, cfgname, [suffix])
    masses = getMasses(prefix, cfgname)
    pprops = getPprops(prefix, cfgname)
    
    sumrule = []

    abovethr = [p[0] for p in widths.query("Out == r'{}'".format(pretty_pname)).index.values]
    sumrule.append(
        8*sp.pi**2*masses.loc[abovethr, "E"]**3\
        /(masses.loc[abovethr, "E"]**2 - masses.loc[pretty_pname, "E"]**2)**3\
        *pprops.loc[abovethr, "xJ"]
    )
    sumrule[0].index = pd.Index([(v, pretty_pname) for v in sumrule[0].index.values])
    
    belowthr = [p[1] for p in widths.query("In == r'{}'".format(pretty_pname)).index.values]
    sumrule.append(
        8*sp.pi**2*masses.loc[pretty_pname, "E"]**3\
        /(masses.loc[pretty_pname, "E"]**2 - masses.loc[belowthr, "E"]**2)**3\
        *pprops.loc[pretty_pname, "xJ"]
    )
    sumrule[1].index = pd.Index([(pretty_pname, v) for v in sumrule[1].index.values])
    sr = pd.concat(sumrule)*(0.19732697)**2*10**4*10**3
    sr = sr.reindex(widths.index.values)
    
    widths["$SR-PDG$"] = widths["PDG"].values
    if sp.any(pd.isna(widths["PDG"])):
        widths.loc[pd.isna(widths["PDG"]), "$SR-PDG$"] = widths.loc[pd.isna(widths["PDG"]).index.values, msuffix+"$\Gamma$"]
    widths.loc[:, "$SR-PDG$"] *= sr*(widths.loc[:, msuffix+"$r^{(2)}$"] - widths.loc[:, msuffix+"$r^{(0)}$"])
    
    widths["$SR-Deng$"] = widths["Deng"]
    if sp.any(pd.isna(widths["Deng"])):
        widths.loc[pd.isna(widths["Deng"]), "$SR-Deng$"] = widths.loc[pd.isna(widths["Deng"]).index.values, msuffix+"$\Gamma$"]
    widths.loc[:, "$SR-Deng$"] *= sr*(widths.loc[:, msuffix+"$r^{(2)}$"] - widths.loc[:, msuffix+"$r^{(0)}$"])
    
    widths["$SR-\Gamma$"] = widths[msuffix+"$\Gamma$"]*sr*(widths.loc[:, msuffix+"$r^{(2)}$"] - widths.loc[:, msuffix+"$r^{(0)}$"])
    
    widths.drop(columns=[msuffix+'$r^{(2)}$', msuffix+'$r^{(0)}$'], inplace=True)
    widths = widths[[widths.columns[i] for i in [0, 3, 1, 4, 2, 5]]]
    widths = dfsort(widths, lambda a,b: cmpMultyStatesByCode(prefix, cfgname
                                                         ,[name2code(i[1:-1]) for i in a]
                                                         ,[name2code(i[1:-1]) for i in b]))
    
    sigma = (widths["$SR-Deng$"] - widths["$SR-\Gamma$"]).std()
    
    return widths, pd.DataFrame(widths[["$SR-PDG$", "$SR-Deng$", "$SR-\Gamma$"]].sum().rename("$SR$")), sigma

In [21]:
def reportSumrules(pname, prefix, cfgname, suffixes=("", "E1", "ELW"), sigma=0):
    if sigma is not None:
        try:
            sigma = list(sigma)
        except TypeError:
            sigma = [sigma]*len(suffixes)
        sigma = dict(zip(suffixes, sigma))
    for approx in suffixes:
        display("Approximation: "+(approx if len(approx) > 0 else "tot"))
        report, sr, autosigma = getSumrule(pname, prefix, cfgname, approx)
        display(sr)
        sigma[approx] = autosigma if sigma[approx] == 0 else sigma[approx]
        display(sigma[approx])
        display(report)
        
        tab = Tabular("|l|l|c|c|c|c|c|c|")
        
        # Header
        tab.add_hline()
        tab.add_row(["", ""] + [NoEscape(i) for i in report.columns])
        tab.add_row([NoEscape(i) for i in report.index.names] + [""]*6)
        tab.add_hline()
        
        curN = "1"
        tot = report.iloc[0, 1::2].fillna(0)*0
        getN = re.compile("\((\d+)[A-Z]\)\$$")
        for entry in report.index.values:
            # Manage subtotals
            label = entry[0] if entry[0] != "${}$".format(code2name(pname)) else entry[1]
            newN = getN.search(label).groups()[0]
            if newN != curN:
                tab.add_hline()
                tab.add_hline()
                try:
                    tab.add_row([MultiColumn(2, align="|c|", data="Subtotal")] +\
                            [MultiColumn(2, align="|r|", data=NoEscape("${:.2f} ({:.2f}\sigma)$".format(num, num/sigma[approx]))) for num in tot])
                except:
                    tab.add_row([MultiColumn(2, align="|c|", data="Subtotal")] +\
                            [MultiColumn(2, align="|r|", data="{:.2f}".format(num)) for num in tot])                
                tab.add_hline()
                tab.add_hline()
                tot = report.loc[entry].fillna(0).values[1::2]
                curN = newN
            else:
                tot += report.loc[entry].fillna(0).values[1::2]
            
            # Regular entry
            tab.add_row([NoEscape(i) for i in list(entry) + ["{:.2f}".format(num) for num in report.loc[entry]]])
            
        # Final subtotal
        tab.add_hline()
        tab.add_hline()
        try:
            tab.add_row([MultiColumn(2, align="|c|", data="Subtotal")] +\
                    [MultiColumn(2, align="|r|", data=NoEscape("${:.2f}~({:.2f}\sigma)$".format(num, num/sigma[approx]))) for num in tot])
        except:
            tab.add_row([MultiColumn(2, align="|c|", data="Subtotal")] +\
                    [MultiColumn(2, align="|r|", data="{:.2f}".format(num)) for num in tot])                
        tab.add_hline()
        tab.add_hline()
        
        # Total
        try:
            tab.add_row([MultiColumn(2, align="|c|", data="Total")] +\
                    [MultiColumn(2, align="|r|", data=NoEscape("${:.2f}~({:.2f}\sigma)$".format(num, num/sigma[approx]))) for num in sr.iloc[:, 0]])
        except:
            tab.add_row([MultiColumn(2, align="|c|", data="Total")] +\
                    [MultiColumn(2, align="|r|", data="{:.2f}".format(num)) for num in sr.iloc[:, 0]])                
        tab.add_hline()
        
        # with open("export/sr{}.{}.{}.tex".format(approx, pname, cfgcode), "w") as f:
        #     f.write(sr.to_latex(escape=False, bold_rows=True, column_format="l|c"))
        # with open("export/sr{}-report.{}.{}.tex".format(approx, pname, cfgcode), "w") as f:
        #     f.write(report.to_latex(escape=False, bold_rows=True, column_format="l|l|c|c|c|c|c|c"))
        tab.generate_tex("export/sr{}-report.{}.{}".format(approx, pname, cfgname))

## Bottomonium sumrules

### $\Upsilon(1S)$

In [22]:
reportSumrules("yps_1S", "scr", "b-scr", sigma=[0]*3)

'Approximation: tot'

,$SR$
$SR-PDG$,-0.648
$SR-Deng$,-1.52
$SR-\Gamma$,-0.648


0.145815571246383

,,PDG,$SR-PDG$,Deng,$SR-Deng$,$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\Upsilon(1S)$,$\eta_{b}(1S)$,nan,-0.542,0.01,-0.505,0.0107,-0.542
$\chi_{b0}(1P)$,$\Upsilon(1S)$,nan,-1.55,27.5,-1.77,24.2,-1.55
$\chi_{b1}(1P)$,$\Upsilon(1S)$,nan,-2.4,31.9,-2.54,30.2,-2.4
$\chi_{b2}(1P)$,$\Upsilon(1S)$,nan,3.7,31.8,3.26,36.1,3.7
$\chi_{b0}(2P)$,$\Upsilon(1S)$,nan,-0.0416,5.54,-0.0519,4.44,-0.0416
$\chi_{b1}(2P)$,$\Upsilon(1S)$,nan,-0.15,10.8,-0.151,10.7,-0.15
$\chi_{b2}(2P)$,$\Upsilon(1S)$,nan,0.3,12.5,0.221,17,0.3
$\chi_{b0}(3P)$,$\Upsilon(1S)$,nan,-0.00561,1.87,-0.00763,1.37,-0.00561
$\chi_{b1}(3P)$,$\Upsilon(1S)$,nan,-0.0355,6.41,-0.0409,5.56,-0.0355


'Approximation: E1'

...

,,PDG,$SR-PDG$,Deng,$SR-Deng$,E1-$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\Upsilon(1S)$,$\eta_{b}(1S)$,nan,-0.542,0.01,-0.505,0.0107,-0.542
$\chi_{b0}(1P)$,$\Upsilon(1S)$,nan,-1.69,27.5,-1.77,26.3,-1.69
$\chi_{b1}(1P)$,$\Upsilon(1S)$,nan,-2.4,31.9,-2.43,31.6,-2.4
$\chi_{b2}(1P)$,$\Upsilon(1S)$,nan,3.85,31.8,3.55,34.5,3.85
$\chi_{b0}(2P)$,$\Upsilon(1S)$,nan,-0.0486,5.54,-0.0519,5.19,-0.0486
$\chi_{b1}(2P)$,$\Upsilon(1S)$,nan,-0.15,10.8,-0.139,11.6,-0.15
$\chi_{b2}(2P)$,$\Upsilon(1S)$,nan,0.322,12.5,0.256,15.7,0.322
$\chi_{b0}(3P)$,$\Upsilon(1S)$,nan,-0.00689,1.87,-0.00763,1.69,-0.00689
$\chi_{b1}(3P)$,$\Upsilon(1S)$,nan,-0.0355,6.41,-0.0371,6.15,-0.0355


'Approximation: ELW'

,$SR$
$SR-PDG$,-0.0587
$SR-Deng$,-0.553
$SR-\Gamma$,-0.0587


0.14386532113364772

,,PDG,$SR-PDG$,Deng,$SR-Deng$,ELW-$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\chi_{b0}(1P)$,$\Upsilon(1S)$,nan,-1.73,27.5,-1.77,26.9,-1.73
$\chi_{b1}(1P)$,$\Upsilon(1S)$,nan,-2.48,31.9,-2.43,32.5,-2.48
$\chi_{b2}(1P)$,$\Upsilon(1S)$,nan,3.98,31.8,3.55,35.7,3.98
$\chi_{b0}(2P)$,$\Upsilon(1S)$,nan,-0.0372,5.54,-0.0519,3.97,-0.0372
$\chi_{b1}(2P)$,$\Upsilon(1S)$,nan,-0.129,10.8,-0.139,10,-0.129
$\chi_{b2}(2P)$,$\Upsilon(1S)$,nan,0.288,12.5,0.256,14.1,0.288
$\chi_{b0}(3P)$,$\Upsilon(1S)$,nan,-0.00414,1.87,-0.00763,1.01,-0.00414
$\chi_{b1}(3P)$,$\Upsilon(1S)$,nan,-0.0279,6.41,-0.0371,4.83,-0.0279
$\chi_{b2}(3P)$,$\Upsilon(1S)$,nan,0.0747,8.17,0.0753,8.1,0.0747


### $\Upsilon(2S)$

In [23]:
reportSumrules("yps_2S", "scr", "b-scr", sigma=[0]*3)

'Approximation: tot'

,$SR$
$SR-PDG$,-1.28
$SR-Deng$,-0.924
$SR-\Gamma$,-0.121


0.36055559857511654

PDG $SR-PDG$  Deng $SR-Deng$  $\Gamma$  \
In              Out                                                         
$\Upsilon(2S)$  $\eta_{b}(1S)$  0.0125 -0.00065 0.066  -0.00344    0.0737   
                $\chi_{b0}(1P)$   1.22    -3.27  1.09     -2.93      1.19   
                $\chi_{b1}(1P)$   2.21    -5.76  2.17     -5.67      2.28   
                $\chi_{b2}(1P)$   2.29     9.99  2.62      11.4      2.58   
$\chi_{b0}(2P)$ $\Upsilon(2S)$     nan    -5.72  14.4     -6.23      13.2   
$\chi_{b1}(2P)$ $\Upsilon(2S)$     nan    -7.45  15.3     -7.47      15.3   
$\chi_{b2}(2P)$ $\Upsilon(2S)$     nan     10.7  15.3      9.85      16.7   
$\chi_{b0}(3P)$ $\Upsilon(2S)$     nan  -0.0872  2.55    -0.102      2.18   
$\chi_{b1}(3P)$ $\Upsilon(2S)$     nan    -0.28  5.63    -0.313      5.04   
$\chi_{b2}(3P)$ $\Upsilon(2S)$     nan    0.548  6.72     0.491      7.51   

                                $SR-\Gamma$  
In              Out                          
$\Upsilon(2S)$  $\eta_{b}(1S)$     -0.00384  
                $\chi_{b0}(1P)$       -3.19  
                $\chi_{b1}(1P)$       -5.94  
                $\chi_{b2}(1P)$        11.3  
$\chi_{b0}(2P)$ $\Upsilon(2S)$        -5.72  
$\chi_{b1}(2P)$ $\Upsilon(2S)$        -7.45  
$\chi_{b2}(2P)$ $\Upsilon(2S)$         10.7  
$\chi_{b0}(3P)$ $\Upsilon(2S)$      -0.0872  
$\chi_{b1}(3P)$ $\Upsilon(2S)$        -0.28  
$\chi_{b2}(3P)$ $\Upsilon(2S)$        0.548

'Approximation: E1'

...

PDG $SR-PDG$  Deng $SR-Deng$  E1-$\Gamma$  \
In              Out                                                            
$\Upsilon(2S)$  $\eta_{b}(1S)$  0.0125 -0.00065 0.066  -0.00344       0.0737   
                $\chi_{b0}(1P)$   1.22    -3.27  1.09     -2.93         1.15   
                $\chi_{b1}(1P)$   2.21    -5.84  2.17     -5.74         2.25   
                $\chi_{b2}(1P)$   2.29     9.79  2.62      11.2         2.61   
$\chi_{b0}(2P)$ $\Upsilon(2S)$     nan    -5.97  14.4     -6.23         13.8   
$\chi_{b1}(2P)$ $\Upsilon(2S)$     nan    -7.45  15.3     -7.29         15.6   
$\chi_{b2}(2P)$ $\Upsilon(2S)$     nan       11  15.3      10.3         16.3   
$\chi_{b0}(3P)$ $\Upsilon(2S)$     nan  -0.0961  2.55    -0.102          2.4   
$\chi_{b1}(3P)$ $\Upsilon(2S)$     nan    -0.28  5.63    -0.298         5.29   
$\chi_{b2}(3P)$ $\Upsilon(2S)$     nan    0.572  6.72     0.539         7.13   

                                $SR-\Gamma$  
In              Out                          
$\Upsilon(2S)$  $\eta_{b}(1S)$     -0.00384  
                $\chi_{b0}(1P)$       -3.08  
                $\chi_{b1}(1P)$       -5.94  
                $\chi_{b2}(1P)$        11.2  
$\chi_{b0}(2P)$ $\Upsilon(2S)$        -5.97  
$\chi_{b1}(2P)$ $\Upsilon(2S)$        -7.45  
$\chi_{b2}(2P)$ $\Upsilon(2S)$           11  
$\chi_{b0}(3P)$ $\Upsilon(2S)$      -0.0961  
$\chi_{b1}(3P)$ $\Upsilon(2S)$        -0.28  
$\chi_{b2}(3P)$ $\Upsilon(2S)$        0.572

'Approximation: ELW'

,$SR$
$SR-PDG$,-1.55
$SR-Deng$,-0.526
$SR-\Gamma$,-0.113


0.42661577948157475

PDG $SR-PDG$  Deng $SR-Deng$  ELW-$\Gamma$  \
In              Out                                                           
$\Upsilon(2S)$  $\chi_{b0}(1P)$ 1.22    -3.27  1.09     -2.93          1.16   
                $\chi_{b1}(1P)$ 2.21    -5.84  2.17     -5.74          2.26   
                $\chi_{b2}(1P)$ 2.29     9.79  2.62      11.2          2.62   
$\chi_{b0}(2P)$ $\Upsilon(2S)$   nan    -6.12  14.4     -6.23          14.2   
$\chi_{b1}(2P)$ $\Upsilon(2S)$   nan     -7.7  15.3     -7.29          16.2   
$\chi_{b2}(2P)$ $\Upsilon(2S)$   nan     11.4  15.3      10.3          16.9   
$\chi_{b0}(3P)$ $\Upsilon(2S)$   nan  -0.0648  2.55    -0.102          1.62   
$\chi_{b1}(3P)$ $\Upsilon(2S)$   nan   -0.224  5.63    -0.298          4.23   
$\chi_{b2}(3P)$ $\Upsilon(2S)$   nan     0.48  6.72     0.539          5.98   

                                $SR-\Gamma$  
In              Out                          
$\Upsilon(2S)$  $\chi_{b0}(1P)$       -3.12  
                $\chi_{b1}(1P)$       -5.98  
                $\chi_{b2}(1P)$        11.2  
$\chi_{b0}(2P)$ $\Upsilon(2S)$        -6.12  
$\chi_{b1}(2P)$ $\Upsilon(2S)$         -7.7  
$\chi_{b2}(2P)$ $\Upsilon(2S)$         11.4  
$\chi_{b0}(3P)$ $\Upsilon(2S)$      -0.0648  
$\chi_{b1}(3P)$ $\Upsilon(2S)$       -0.224  
$\chi_{b2}(3P)$ $\Upsilon(2S)$         0.48

### $\Upsilon(3S)$

In [24]:
reportSumrules("yps_3S", "scr", "b-scr")

'Approximation: tot'

,$SR$
$SR-PDG$,-3.93
$SR-Deng$,-2.47
$SR-\Gamma$,-0.502


0.5568848022586923

PDG  $SR-PDG$   Deng $SR-Deng$  $\Gamma$  \
In              Out                                                           
$\Upsilon(3S)$  $\eta_{b}(1S)$  0.0104 -0.000157  0.071  -0.00108    0.0795   
                $\chi_{b0}(1P)$ 0.0549  -0.00558  0.097  -0.00987     0.116   
                $\chi_{b1}(1P)$ 0.0183  -0.00109 0.0005 -2.98e-05  9.28e-05   
                $\chi_{b2}(1P)$  0.201    0.0154   0.14    0.0107     0.153   
                $\eta_{b}(2S)$     nan  -0.00327  0.011  -0.00296    0.0122   
                $\chi_{b0}(2P)$    1.2     -7.62   1.21     -7.69      1.31   
                $\chi_{b1}(2P)$   2.56     -14.9   2.61     -15.2      2.66   
                $\chi_{b2}(2P)$   2.66      24.4   3.16      28.9      3.18   
$\chi_{b0}(3P)$ $\Upsilon(3S)$     nan     -11.8   7.95     -12.3      7.59   
$\chi_{b1}(3P)$ $\Upsilon(3S)$     nan     -14.6   10.3       -16      9.38   
$\chi_{b2}(3P)$ $\Upsilon(3S)$     nan      20.6   10.8      19.9      11.2   

                                $SR-\Gamma$  
In              Out                          
$\Upsilon(3S)$  $\eta_{b}(1S)$     -0.00121  
                $\chi_{b0}(1P)$     -0.0118  
                $\chi_{b1}(1P)$   -5.53e-06  
                $\chi_{b2}(1P)$      0.0117  
                $\eta_{b}(2S)$     -0.00327  
                $\chi_{b0}(2P)$       -8.35  
                $\chi_{b1}(2P)$       -15.5  
                $\chi_{b2}(2P)$        29.1  
$\chi_{b0}(3P)$ $\Upsilon(3S)$        -11.8  
$\chi_{b1}(3P)$ $\Upsilon(3S)$        -14.6  
$\chi_{b2}(3P)$ $\Upsilon(3S)$         20.6

'Approximation: E1'

...

PDG  $SR-PDG$   Deng $SR-Deng$  \
In              Out                                                 
$\Upsilon(3S)$  $\eta_{b}(1S)$  0.0104 -0.000157  0.071  -0.00108   
                $\chi_{b0}(1P)$ 0.0549  -0.00558  0.097  -0.00987   
                $\chi_{b1}(1P)$ 0.0183  -0.00114 0.0005 -3.12e-05   
                $\chi_{b2}(1P)$  0.201    0.0142   0.14   0.00991   
                $\eta_{b}(2S)$     nan  -0.00327  0.011  -0.00296   
                $\chi_{b0}(2P)$    1.2     -7.62   1.21     -7.69   
                $\chi_{b1}(2P)$   2.56     -15.1   2.61     -15.4   
                $\chi_{b2}(2P)$   2.66        24   3.16      28.5   
$\chi_{b0}(3P)$ $\Upsilon(3S)$     nan     -12.1   7.95     -12.3   
$\chi_{b1}(3P)$ $\Upsilon(3S)$     nan     -14.6   10.3     -15.8   
$\chi_{b2}(3P)$ $\Upsilon(3S)$     nan      20.9   10.8      20.5   

                                 E1-$\Gamma$ $SR-\Gamma$  
In              Out                                       
$\Upsilon(3S)$  $\eta_{b}(1S)$        0.0795    -0.00121  
                $\chi_{b0}(1P)$        0.105     -0.0107  
                $\chi_{b1}(1P)$     8.86e-05   -5.53e-06  
                $\chi_{b2}(1P)$         0.16      0.0113  
                $\eta_{b}(2S)$        0.0122    -0.00327  
                $\chi_{b0}(2P)$         1.28       -8.14  
                $\chi_{b1}(2P)$         2.63       -15.5  
                $\chi_{b2}(2P)$         3.21        28.9  
$\chi_{b0}(3P)$ $\Upsilon(3S)$          7.81       -12.1  
$\chi_{b1}(3P)$ $\Upsilon(3S)$          9.54       -14.6  
$\chi_{b2}(3P)$ $\Upsilon(3S)$            11        20.9

'Approximation: ELW'

,$SR$
$SR-PDG$,-4.38
$SR-Deng$,-2.17
$SR-\Gamma$,-0.436


0.5617698084348861

PDG $SR-PDG$   Deng $SR-Deng$  \
In              Out                                                
$\Upsilon(3S)$  $\chi_{b0}(1P)$ 0.0549 -0.00558  0.097  -0.00987   
                $\chi_{b1}(1P)$ 0.0183 -0.00114 0.0005 -3.12e-05   
                $\chi_{b2}(1P)$  0.201   0.0142   0.14   0.00991   
                $\chi_{b0}(2P)$    1.2    -7.62   1.21     -7.69   
                $\chi_{b1}(2P)$   2.56    -15.1   2.61     -15.4   
                $\chi_{b2}(2P)$   2.66       24   3.16      28.5   
$\chi_{b0}(3P)$ $\Upsilon(3S)$     nan    -12.4   7.95     -12.3   
$\chi_{b1}(3P)$ $\Upsilon(3S)$     nan    -15.1   10.3     -15.8   
$\chi_{b2}(3P)$ $\Upsilon(3S)$     nan     21.8   10.8      20.5   

                                 ELW-$\Gamma$ $SR-\Gamma$  
In              Out                                        
$\Upsilon(3S)$  $\chi_{b0}(1P)$        0.0436    -0.00443  
                $\chi_{b1}(1P)$        0.0413    -0.00257  
                $\chi_{b2}(1P)$         0.455      0.0322  
                $\chi_{b0}(2P)$           1.3       -8.25  
                $\chi_{b1}(2P)$          2.65       -15.6  
                $\chi_{b2}(2P)$          3.23        29.1  
$\chi_{b0}(3P)$ $\Upsilon(3S)$           7.98       -12.4  
$\chi_{b1}(3P)$ $\Upsilon(3S)$           9.84       -15.1  
$\chi_{b2}(3P)$ $\Upsilon(3S)$           11.5        21.8

## Charmonium sumrules

### $\psi(1S)$

In [25]:
reportSumrules("psi_1S", "scr", "c-scr", sigma=6.7)

'Approximation: tot'

,$SR$
$SR-PDG$,-30.5
$SR-Deng$,-54.8
$SR-\Gamma$,-29.4


6.7

,,PDG,$SR-PDG$,Deng,$SR-Deng$,$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,-13.3,2.44,-20.6,2.79,-23.6
$\chi_{c0}(1P)$,$\psi(1S)$,151,-20.9,179,-24.7,142,-19.6
$\chi_{c1}(1P)$,$\psi(1S)$,288,-32,319,-35.4,310,-34.5
$\chi_{c2}(1P)$,$\psi(1S)$,374,34.1,292,26.6,510,46.5
$\eta_{c}(2S)$,$\psi(1S)$,nan,-0.0893,2.29,-0.0702,2.91,-0.0893
$\chi_{c0}(2P)$,$\psi(1S)$,nan,-0.0222,2.3,-0.027,1.89,-0.0222
$\chi_{c1}(2P)$,$\psi(1S)$,nan,-1.58,88,-1.86,74.4,-1.58
$\chi_{c2}(2P)$,$\psi(1S)$,nan,3.36,93,1.32,237,3.36


'Approximation: E1'

...

,,PDG,$SR-PDG$,Deng,$SR-Deng$,E1-$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,-13.3,2.44,-20.6,2.79,-23.6
$\chi_{c0}(1P)$,$\psi(1S)$,151,-20.9,179,-24.7,176,-24.3
$\chi_{c1}(1P)$,$\psi(1S)$,288,-28.1,319,-31.1,353,-34.5
$\chi_{c2}(1P)$,$\psi(1S)$,374,45.4,292,35.4,441,53.5
$\eta_{c}(2S)$,$\psi(1S)$,nan,-0.0893,2.29,-0.0702,2.91,-0.0893
$\chi_{c0}(2P)$,$\psi(1S)$,nan,-0.0365,2.3,-0.027,3.11,-0.0365
$\chi_{c1}(2P)$,$\psi(1S)$,nan,-1.58,88,-1.5,92.7,-1.58
$\chi_{c2}(2P)$,$\psi(1S)$,nan,4.37,93,2.18,186,4.37


'Approximation: ELW'

,$SR$
$SR-PDG$,-1.49
$SR-Deng$,-19.8
$SR-\Gamma$,-1.12


6.7

,,PDG,$SR-PDG$,Deng,$SR-Deng$,ELW-$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\chi_{c0}(1P)$,$\psi(1S)$,151,-20.9,179,-24.7,183,-25.3
$\chi_{c1}(1P)$,$\psi(1S)$,288,-28.1,319,-31.1,382,-37.3
$\chi_{c2}(1P)$,$\psi(1S)$,374,45.4,292,35.4,490,59.3
$\chi_{c0}(2P)$,$\psi(1S)$,nan,-0.0387,2.3,-0.027,3.3,-0.0387
$\chi_{c1}(2P)$,$\psi(1S)$,nan,-0.883,88,-1.5,51.9,-0.883
$\chi_{c2}(2P)$,$\psi(1S)$,nan,3.09,93,2.18,132,3.09


In [26]:
reportSumrules("psi_1S", "lin", "c-lin", sigma=6.7)

'Approximation: tot'

,$SR$
$SR-PDG$,-29.6
$SR-Deng$,-51.9
$SR-\Gamma$,-26.6


6.7

,,PDG,$SR-PDG$,Deng,$SR-Deng$,$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,-13.3,2.39,-20.2,2.53,-21.3
$\chi_{c0}(1P)$,$\psi(1S)$,151,-20.9,172,-23.8,138,-19.1
$\chi_{c1}(1P)$,$\psi(1S)$,288,-31.8,306,-33.8,301,-33.3
$\chi_{c2}(1P)$,$\psi(1S)$,374,34.6,284,26.3,490,45.3
$\eta_{c}(2S)$,$\psi(1S)$,nan,-0.093,2.64,-0.0809,3.04,-0.093
$\chi_{c0}(2P)$,$\psi(1S)$,nan,-0.0313,6.1,-0.0716,2.66,-0.0313
$\chi_{c1}(2P)$,$\psi(1S)$,nan,-1.45,81,-1.7,69.1,-1.45
$\chi_{c2}(2P)$,$\psi(1S)$,nan,3.37,93,1.36,231,3.37


'Approximation: E1'

...

,,PDG,$SR-PDG$,Deng,$SR-Deng$,E1-$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,-13.3,2.39,-20.2,2.53,-21.3
$\chi_{c0}(1P)$,$\psi(1S)$,151,-20.9,172,-23.8,170,-23.4
$\chi_{c1}(1P)$,$\psi(1S)$,288,-28.1,306,-29.9,341,-33.3
$\chi_{c2}(1P)$,$\psi(1S)$,374,45.4,284,34.4,427,51.7
$\eta_{c}(2S)$,$\psi(1S)$,nan,-0.093,2.64,-0.0809,3.04,-0.093
$\chi_{c0}(2P)$,$\psi(1S)$,nan,-0.05,6.1,-0.0716,4.26,-0.05
$\chi_{c1}(2P)$,$\psi(1S)$,nan,-1.45,81,-1.38,85.2,-1.45
$\chi_{c2}(2P)$,$\psi(1S)$,nan,4.32,93,2.18,184,4.32


'Approximation: ELW'

,$SR$
$SR-PDG$,-1.45
$SR-Deng$,-18.5
$SR-\Gamma$,-0.975


6.7

,,PDG,$SR-PDG$,Deng,$SR-Deng$,ELW-$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\chi_{c0}(1P)$,$\psi(1S)$,151,-20.9,172,-23.8,177,-24.4
$\chi_{c1}(1P)$,$\psi(1S)$,288,-28.1,306,-29.9,367,-35.9
$\chi_{c2}(1P)$,$\psi(1S)$,374,45.4,284,34.4,471,57.1
$\chi_{c0}(2P)$,$\psi(1S)$,nan,-0.0195,6.1,-0.0716,1.66,-0.0195
$\chi_{c1}(2P)$,$\psi(1S)$,nan,-0.789,81,-1.38,46.4,-0.789
$\chi_{c2}(2P)$,$\psi(1S)$,nan,3.02,93,2.18,129,3.02


### $\psi(2S)$

In [27]:
reportSumrules("psi_2S", "scr", "c-scr", sigma=16.6)

'Approximation: tot'

,$SR$
$SR-PDG$,-41.1
$SR-Deng$,-83.8
$SR-\Gamma$,-29.4


16.6

PDG $SR-PDG$  Deng $SR-Deng$  $\Gamma$  \
In              Out                                                        
$\psi(2S)$      $\eta_{c}(1S)$      1  -0.0449   7.8    -0.351      10.6   
                $\chi_{c0}(1P)$  28.8    -18.6    22     -14.2      36.6   
                $\chi_{c1}(1P)$  28.7      -31    45     -48.7        56   
                $\chi_{c2}(1P)$    28     83.8    46       138      44.5   
                $\eta_{c}(2S)$  0.206    -21.2  0.19     -19.6     0.233   
$\chi_{c0}(2P)$ $\psi(2S)$        nan    -81.4    99     -74.9       108   
$\chi_{c1}(2P)$ $\psi(2S)$        nan     -105   155      -160       102   
$\chi_{c2}(2P)$ $\psi(2S)$        nan      129   150      96.9       200   
$\chi_{c0}(3P)$ $\psi(2S)$        nan   -0.309   9.1    -0.425      6.62   
$\chi_{c1}(3P)$ $\psi(2S)$        nan    -4.27    74     -4.65      68.1   
$\chi_{c2}(3P)$ $\psi(2S)$        nan     7.86    76      4.41       136   

                                $SR-\Gamma$  
In              Out                          
$\psi(2S)$      $\eta_{c}(1S)$       -0.479  
                $\chi_{c0}(1P)$       -23.7  
                $\chi_{c1}(1P)$       -60.6  
                $\chi_{c2}(1P)$         133  
                $\eta_{c}(2S)$          -24  
$\chi_{c0}(2P)$ $\psi(2S)$            -81.4  
$\chi_{c1}(2P)$ $\psi(2S)$             -105  
$\chi_{c2}(2P)$ $\psi(2S)$              129  
$\chi_{c0}(3P)$ $\psi(2S)$           -0.309  
$\chi_{c1}(3P)$ $\psi(2S)$            -4.27  
$\chi_{c2}(3P)$ $\psi(2S)$             7.86

'Approximation: E1'

...

PDG $SR-PDG$  Deng $SR-Deng$  E1-$\Gamma$  \
In              Out                                                           
$\psi(2S)$      $\eta_{c}(1S)$      1  -0.0449   7.8    -0.351         10.6   
                $\chi_{c0}(1P)$  28.8    -18.6    22     -14.2         30.9   
                $\chi_{c1}(1P)$  28.7    -32.9    45     -51.6         52.8   
                $\chi_{c2}(1P)$    28     77.6    46       128         46.5   
                $\eta_{c}(2S)$  0.206    -21.2  0.19     -19.6        0.233   
$\chi_{c0}(2P)$ $\psi(2S)$        nan      -92    99     -74.9          122   
$\chi_{c1}(2P)$ $\psi(2S)$        nan     -105   155      -150          109   
$\chi_{c2}(2P)$ $\psi(2S)$        nan      139   150       113          185   
$\chi_{c0}(3P)$ $\psi(2S)$        nan   -0.424   9.1    -0.425         9.07   
$\chi_{c1}(3P)$ $\psi(2S)$        nan    -4.27    74     -3.97         79.7   
$\chi_{c2}(3P)$ $\psi(2S)$        nan     9.27    76      6.14          115   

                                $SR-\Gamma$  
In              Out                          
$\psi(2S)$      $\eta_{c}(1S)$       -0.479  
                $\chi_{c0}(1P)$       -19.9  
                $\chi_{c1}(1P)$       -60.6  
                $\chi_{c2}(1P)$         129  
                $\eta_{c}(2S)$          -24  
$\chi_{c0}(2P)$ $\psi(2S)$              -92  
$\chi_{c1}(2P)$ $\psi(2S)$             -105  
$\chi_{c2}(2P)$ $\psi(2S)$              139  
$\chi_{c0}(3P)$ $\psi(2S)$           -0.424  
$\chi_{c1}(3P)$ $\psi(2S)$            -4.27  
$\chi_{c2}(3P)$ $\psi(2S)$             9.27

'Approximation: ELW'

,$SR$
$SR-PDG$,-22.9
$SR-Deng$,-48.9
$SR-\Gamma$,-1.84


16.6

PDG $SR-PDG$  Deng $SR-Deng$  ELW-$\Gamma$  \
In              Out                                                           
$\psi(2S)$      $\chi_{c0}(1P)$ 28.8    -18.6    22     -14.2          33.5   
                $\chi_{c1}(1P)$ 28.7    -32.9    45     -51.6          54.7   
                $\chi_{c2}(1P)$   28     77.6    46       128          47.5   
$\chi_{c0}(2P)$ $\psi(2S)$       nan    -96.7    99     -74.9           128   
$\chi_{c1}(2P)$ $\psi(2S)$       nan     -112   155      -150           116   
$\chi_{c2}(2P)$ $\psi(2S)$       nan      156   150       113           207   
$\chi_{c0}(3P)$ $\psi(2S)$       nan -0.00185   9.1    -0.425        0.0396   
$\chi_{c1}(3P)$ $\psi(2S)$       nan    -2.15    74     -3.97          40.1   
$\chi_{c2}(3P)$ $\psi(2S)$       nan     6.07    76      6.14          75.1   

                                $SR-\Gamma$  
In              Out                          
$\psi(2S)$      $\chi_{c0}(1P)$       -21.6  
                $\chi_{c1}(1P)$       -62.8  
                $\chi_{c2}(1P)$         132  
$\chi_{c0}(2P)$ $\psi(2S)$            -96.7  
$\chi_{c1}(2P)$ $\psi(2S)$             -112  
$\chi_{c2}(2P)$ $\psi(2S)$              156  
$\chi_{c0}(3P)$ $\psi(2S)$         -0.00185  
$\chi_{c1}(3P)$ $\psi(2S)$            -2.15  
$\chi_{c2}(3P)$ $\psi(2S)$             6.07

In [28]:
reportSumrules("psi_2S", "lin", "c-lin", sigma=16.6)

'Approximation: tot'

,$SR$
$SR-PDG$,-34.5
$SR-Deng$,-114
$SR-\Gamma$,-25.3


16.6

PDG $SR-PDG$  Deng $SR-Deng$  $\Gamma$  \
In              Out                                                        
$\psi(2S)$      $\eta_{c}(1S)$      1  -0.0449  8.08    -0.363      9.98   
                $\chi_{c0}(1P)$  28.8    -18.6    22     -14.2      37.2   
                $\chi_{c1}(1P)$  28.7    -31.1    42     -45.6      52.2   
                $\chi_{c2}(1P)$    28     83.5    38       113      41.8   
                $\eta_{c}(2S)$  0.206    -21.2  0.19     -19.6     0.211   
$\chi_{c0}(2P)$ $\psi(2S)$        nan    -71.4   121     -91.5      94.4   
$\chi_{c1}(2P)$ $\psi(2S)$        nan    -93.7   139      -143        91   
$\chi_{c2}(2P)$ $\psi(2S)$        nan      115   135      87.9       177   
$\chi_{c0}(3P)$ $\psi(2S)$        nan   -0.302    17    -0.493      10.4   
$\chi_{c1}(3P)$ $\psi(2S)$        nan    -3.45    94     -3.74      86.6   
$\chi_{c2}(3P)$ $\psi(2S)$        nan     6.57    97      3.31       192   

                                $SR-\Gamma$  
In              Out                          
$\psi(2S)$      $\eta_{c}(1S)$       -0.449  
                $\chi_{c0}(1P)$         -24  
                $\chi_{c1}(1P)$       -56.6  
                $\chi_{c2}(1P)$         125  
                $\eta_{c}(2S)$        -21.7  
$\chi_{c0}(2P)$ $\psi(2S)$            -71.4  
$\chi_{c1}(2P)$ $\psi(2S)$            -93.7  
$\chi_{c2}(2P)$ $\psi(2S)$              115  
$\chi_{c0}(3P)$ $\psi(2S)$           -0.302  
$\chi_{c1}(3P)$ $\psi(2S)$            -3.45  
$\chi_{c2}(3P)$ $\psi(2S)$             6.57

'Approximation: E1'

...

PDG $SR-PDG$  Deng $SR-Deng$  E1-$\Gamma$  \
In              Out                                                           
$\psi(2S)$      $\eta_{c}(1S)$      1  -0.0449  8.08    -0.363         9.98   
                $\chi_{c0}(1P)$  28.8    -18.6    22     -14.2         31.6   
                $\chi_{c1}(1P)$  28.7    -32.9    42     -48.2         49.3   
                $\chi_{c2}(1P)$    28     77.6    38       105         43.5   
                $\eta_{c}(2S)$  0.206    -21.2  0.19     -19.6        0.211   
$\chi_{c0}(2P)$ $\psi(2S)$        nan    -80.3   121     -91.5          106   
$\chi_{c1}(2P)$ $\psi(2S)$        nan    -93.7   139      -135         96.6   
$\chi_{c2}(2P)$ $\psi(2S)$        nan      123   135       102          164   
$\chi_{c0}(3P)$ $\psi(2S)$        nan   -0.429    17    -0.493         14.8   
$\chi_{c1}(3P)$ $\psi(2S)$        nan    -3.45    94     -3.15          103   
$\chi_{c2}(3P)$ $\psi(2S)$        nan     7.92    97      4.81          160   

                                $SR-\Gamma$  
In              Out                          
$\psi(2S)$      $\eta_{c}(1S)$       -0.449  
                $\chi_{c0}(1P)$       -20.4  
                $\chi_{c1}(1P)$       -56.6  
                $\chi_{c2}(1P)$         121  
                $\eta_{c}(2S)$        -21.7  
$\chi_{c0}(2P)$ $\psi(2S)$            -80.3  
$\chi_{c1}(2P)$ $\psi(2S)$            -93.7  
$\chi_{c2}(2P)$ $\psi(2S)$              123  
$\chi_{c0}(3P)$ $\psi(2S)$           -0.429  
$\chi_{c1}(3P)$ $\psi(2S)$            -3.45  
$\chi_{c2}(3P)$ $\psi(2S)$             7.92

'Approximation: ELW'

,$SR$
$SR-PDG$,-17.5
$SR-Deng$,-80.6
$SR-\Gamma$,-1.12


16.6

PDG $SR-PDG$  Deng $SR-Deng$  ELW-$\Gamma$  \
In              Out                                                           
$\psi(2S)$      $\chi_{c0}(1P)$ 28.8    -18.6    22     -14.2          34.1   
                $\chi_{c1}(1P)$ 28.7    -32.9    42     -48.2            51   
                $\chi_{c2}(1P)$   28     77.6    38       105          44.4   
$\chi_{c0}(2P)$ $\psi(2S)$       nan    -83.9   121     -91.5           111   
$\chi_{c1}(2P)$ $\psi(2S)$       nan      -99   139      -135           102   
$\chi_{c2}(2P)$ $\psi(2S)$       nan      136   135       102           181   
$\chi_{c0}(3P)$ $\psi(2S)$       nan  -0.0135    17    -0.493         0.466   
$\chi_{c1}(3P)$ $\psi(2S)$       nan    -1.26    94     -3.15          37.5   
$\chi_{c2}(3P)$ $\psi(2S)$       nan      4.3    97      4.81          86.7   

                                $SR-\Gamma$  
In              Out                          
$\psi(2S)$      $\chi_{c0}(1P)$         -22  
                $\chi_{c1}(1P)$       -58.5  
                $\chi_{c2}(1P)$         123  
$\chi_{c0}(2P)$ $\psi(2S)$            -83.9  
$\chi_{c1}(2P)$ $\psi(2S)$              -99  
$\chi_{c2}(2P)$ $\psi(2S)$              136  
$\chi_{c0}(3P)$ $\psi(2S)$          -0.0135  
$\chi_{c1}(3P)$ $\psi(2S)$            -1.26  
$\chi_{c2}(3P)$ $\psi(2S)$              4.3

### $\psi(3S)$

In [29]:
reportSumrules("psi_3S", "scr", "c-scr")

'Approximation: tot'

,$SR$
$SR-PDG$,103
$SR-Deng$,209
$SR-\Gamma$,103


26.390715813467384

PDG  $SR-PDG$  Deng  $SR-Deng$  $\Gamma$  \
In         Out                                                         
$\psi(3S)$ $\eta_{c}(1S)$   nan    -0.149  6.76     -0.101      9.95   
           $\chi_{c0}(1P)$  nan     -1.24   6.7     -0.404      20.6   
           $\chi_{c1}(1P)$  nan    -0.488   6.7     -0.273        12   
           $\chi_{c2}(1P)$  nan     0.171   2.5      0.196      2.18   
           $\eta_{c}(2S)$   nan    -0.458  1.78      -0.37       2.2   
           $\chi_{c0}(2P)$  nan     -67.9    27        -60      30.6   
           $\chi_{c1}(2P)$  nan      -163    67        -83       132   
           $\chi_{c2}(2P)$  nan       361    82        377      78.4   
           $\eta_{c}(3S)$   nan     -24.2 0.088      -24.5     0.087   

                            $SR-\Gamma$  
In         Out                           
$\psi(3S)$ $\eta_{c}(1S)$        -0.149  
           $\chi_{c0}(1P)$        -1.24  
           $\chi_{c1}(1P)$       -0.488  
           $\chi_{c2}(1P)$        0.171  
           $\eta_{c}(2S)$        -0.458  
           $\chi_{c0}(2P)$        -67.9  
           $\chi_{c1}(2P)$         -163  
           $\chi_{c2}(2P)$          361  
           $\eta_{c}(3S)$         -24.2

'Approximation: E1'

...

PDG  $SR-PDG$  Deng  $SR-Deng$  E1-$\Gamma$  \
In         Out                                                            
$\psi(3S)$ $\eta_{c}(1S)$   nan    -0.149  6.76     -0.101         9.95   
           $\chi_{c0}(1P)$  nan    -0.879   6.7     -0.404         14.6   
           $\chi_{c1}(1P)$  nan    -0.488   6.7     -0.321         10.2   
           $\chi_{c2}(1P)$  nan     0.155   2.5      0.154         2.52   
           $\eta_{c}(2S)$   nan    -0.458  1.78      -0.37          2.2   
           $\chi_{c0}(2P)$  nan     -60.4    27        -60         27.2   
           $\chi_{c1}(2P)$  nan      -163    67      -87.8          125   
           $\chi_{c2}(2P)$  nan       350    82        353         81.4   
           $\eta_{c}(3S)$   nan     -24.2 0.088      -24.5        0.087   

                            $SR-\Gamma$  
In         Out                           
$\psi(3S)$ $\eta_{c}(1S)$        -0.149  
           $\chi_{c0}(1P)$       -0.879  
           $\chi_{c1}(1P)$       -0.488  
           $\chi_{c2}(1P)$        0.155  
           $\eta_{c}(2S)$        -0.458  
           $\chi_{c0}(2P)$        -60.4  
           $\chi_{c1}(2P)$         -163  
           $\chi_{c2}(2P)$          350  
           $\eta_{c}(3S)$         -24.2

'Approximation: ELW'

,$SR$
$SR-PDG$,119
$SR-Deng$,204
$SR-\Gamma$,119


37.165665490716144

PDG  $SR-PDG$  Deng  $SR-Deng$  ELW-$\Gamma$  \
In         Out                                                             
$\psi(3S)$ $\chi_{c0}(1P)$  nan    -0.491   6.7     -0.404          8.15   
           $\chi_{c1}(1P)$  nan   -0.0361   6.7     -0.321         0.753   
           $\chi_{c2}(1P)$  nan     0.123   2.5      0.154             2   
           $\chi_{c0}(2P)$  nan     -66.3    27        -60          29.8   
           $\chi_{c1}(2P)$  nan      -177    67      -87.8           135   
           $\chi_{c2}(2P)$  nan       363    82        353          84.4   

                            $SR-\Gamma$  
In         Out                           
$\psi(3S)$ $\chi_{c0}(1P)$       -0.491  
           $\chi_{c1}(1P)$      -0.0361  
           $\chi_{c2}(1P)$        0.123  
           $\chi_{c0}(2P)$        -66.3  
           $\chi_{c1}(2P)$         -177  
           $\chi_{c2}(2P)$          363

In [30]:
reportSumrules("psi_3S", "lin", "c-lin")

'Approximation: tot'

,$SR$
$SR-PDG$,96.6
$SR-Deng$,196
$SR-\Gamma$,96.6


22.559134976276596

PDG  $SR-PDG$  Deng  $SR-Deng$  $\Gamma$  \
In         Out                                                         
$\psi(3S)$ $\eta_{c}(1S)$   nan    -0.135  6.66    -0.0995      9.06   
           $\chi_{c0}(1P)$  nan     -1.04   5.9     -0.356      17.3   
           $\chi_{c1}(1P)$  nan    -0.287     4     -0.164         7   
           $\chi_{c2}(1P)$  nan    0.0115  0.25     0.0194     0.148   
           $\eta_{c}(2S)$   nan    -0.385  1.65     -0.343      1.85   
           $\chi_{c0}(2P)$  nan     -60.4    19      -42.2      27.2   
           $\chi_{c1}(2P)$  nan      -133    55      -68.3       107   
           $\chi_{c2}(2P)$  nan       292    67        307      63.7   

                            $SR-\Gamma$  
In         Out                           
$\psi(3S)$ $\eta_{c}(1S)$        -0.135  
           $\chi_{c0}(1P)$        -1.04  
           $\chi_{c1}(1P)$       -0.287  
           $\chi_{c2}(1P)$       0.0115  
           $\eta_{c}(2S)$        -0.385  
           $\chi_{c0}(2P)$        -60.4  
           $\chi_{c1}(2P)$         -133  
           $\chi_{c2}(2P)$          292

'Approximation: E1'

...

PDG  $SR-PDG$  Deng  $SR-Deng$  E1-$\Gamma$  \
In         Out                                                            
$\psi(3S)$ $\eta_{c}(1S)$   nan    -0.135  6.66    -0.0995         9.06   
           $\chi_{c0}(1P)$  nan    -0.749   5.9     -0.356         12.4   
           $\chi_{c1}(1P)$  nan    -0.287     4     -0.192         5.99   
           $\chi_{c2}(1P)$  nan    0.0105  0.25     0.0154         0.17   
           $\eta_{c}(2S)$   nan    -0.385  1.65     -0.343         1.85   
           $\chi_{c0}(2P)$  nan     -54.1    19      -42.2         24.3   
           $\chi_{c1}(2P)$  nan      -133    55      -72.1          102   
           $\chi_{c2}(2P)$  nan       284    67        288         66.1   

                            $SR-\Gamma$  
In         Out                           
$\psi(3S)$ $\eta_{c}(1S)$        -0.135  
           $\chi_{c0}(1P)$       -0.749  
           $\chi_{c1}(1P)$       -0.287  
           $\chi_{c2}(1P)$       0.0105  
           $\eta_{c}(2S)$        -0.385  
           $\chi_{c0}(2P)$        -54.1  
           $\chi_{c1}(2P)$         -133  
           $\chi_{c2}(2P)$          284

'Approximation: ELW'

,$SR$
$SR-PDG$,91.8
$SR-Deng$,173
$SR-\Gamma$,91.8


28.71714994909823

PDG  $SR-PDG$  Deng  $SR-Deng$  ELW-$\Gamma$  \
In         Out                                                             
$\psi(3S)$ $\chi_{c0}(1P)$  nan    -0.375   5.9     -0.356          6.21   
           $\chi_{c1}(1P)$  nan  -0.00421     4     -0.192        0.0879   
           $\chi_{c2}(1P)$  nan     0.343  0.25     0.0154          5.56   
           $\chi_{c0}(2P)$  nan     -58.4    19      -42.2          26.3   
           $\chi_{c1}(2P)$  nan      -142    55      -72.1           109   
           $\chi_{c2}(2P)$  nan       293    67        288            68   

                            $SR-\Gamma$  
In         Out                           
$\psi(3S)$ $\chi_{c0}(1P)$       -0.375  
           $\chi_{c1}(1P)$     -0.00421  
           $\chi_{c2}(1P)$        0.343  
           $\chi_{c0}(2P)$        -58.4  
           $\chi_{c1}(2P)$         -142  
           $\chi_{c2}(2P)$          293

In [31]:
reportSumrules("psi_1_1D", "scr", "c-scr", sigma=[0]*3)

'Approximation: tot'

,$SR$
$SR-PDG$,-264
$SR-Deng$,-268
$SR-\Gamma$,-299


24.032366835470626

PDG  $SR-PDG$  Deng $SR-Deng$  $\Gamma$  \
In              Out                                                        
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan   -0.0124   nan   -0.0124     0.386   
                $\chi_{c0}(1P)$  188     -52.3   261     -72.7       253   
                $\chi_{c1}(1P)$ 67.7     -24.5   135     -48.8       115   
                $\chi_{c2}(1P)$  nan      1.65   8.1      2.82      4.73   
                $\eta_{c}(2S)$   nan -0.000483   nan -0.000483   0.00011   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan      -151    12     -80.7      22.5   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan     -40.3   9.8     -69.7      5.67   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan      2.87  0.46      1.63     0.812   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan   -0.0394  0.39   -0.0343     0.448   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan    -0.363     2    -0.167      4.35   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan   -0.0534  0.79   -0.0281       1.5   

                                $SR-\Gamma$  
In              Out                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$      -0.0124  
                $\chi_{c0}(1P)$       -70.5  
                $\chi_{c1}(1P)$       -41.6  
                $\chi_{c2}(1P)$        1.65  
                $\eta_{c}(2S)$    -0.000483  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$         -151  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$        -40.3  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$         2.87  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$      -0.0394  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$       -0.363  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$      -0.0534

'Approximation: E1'

...

PDG  $SR-PDG$  Deng $SR-Deng$  E1-$\Gamma$  \
In              Out                                                           
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan   -0.0124   nan   -0.0124        0.386   
                $\chi_{c0}(1P)$  188     -52.3   261     -72.7          285   
                $\chi_{c1}(1P)$ 67.7     -22.7   135     -45.3          138   
                $\chi_{c2}(1P)$  nan      3.63   8.1      4.67         6.29   
                $\eta_{c}(2S)$   nan -0.000483   nan -0.000483      0.00011   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan      -147    12     -80.7         21.8   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan     -38.8   9.8     -71.5         5.32   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan      2.02  0.46      1.41        0.656   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan   -0.0351  0.39   -0.0343        0.399   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan     -0.32     2    -0.189         3.38   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan    0.0349  0.79      0.11         0.25   

                                $SR-\Gamma$  
In              Out                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$      -0.0124  
                $\chi_{c0}(1P)$       -79.5  
                $\chi_{c1}(1P)$       -46.4  
                $\chi_{c2}(1P)$        3.63  
                $\eta_{c}(2S)$    -0.000483  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$         -147  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$        -38.8  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$         2.02  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$      -0.0351  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$        -0.32  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$       0.0349

'Approximation: ELW'

,$SR$
$SR-PDG$,-258
$SR-Deng$,-264
$SR-\Gamma$,-321


26.309758527106396

PDG $SR-PDG$  Deng $SR-Deng$  ELW-$\Gamma$  \
In              Out                                                           
$\psi_{1}(1D)$  $\chi_{c0}(1P)$  188    -52.3   261     -72.7           318   
                $\chi_{c1}(1P)$ 67.7    -22.7   135     -45.3           148   
                $\chi_{c2}(1P)$  nan     3.81   8.1      4.67          6.61   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan     -149    12     -80.7          22.1   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan    -39.6   9.8     -71.5          5.42   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan     2.12  0.46      1.41         0.692   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan   -0.835  0.39   -0.0343          9.49   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan  -0.0371     2    -0.189         0.392   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan    0.011  0.79      0.11        0.0791   

                                $SR-\Gamma$  
In              Out                          
$\psi_{1}(1D)$  $\chi_{c0}(1P)$       -88.5  
                $\chi_{c1}(1P)$       -49.6  
                $\chi_{c2}(1P)$        3.81  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$         -149  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$        -39.6  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$         2.12  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$       -0.835  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$      -0.0371  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$        0.011

In [32]:
reportSumrules("psi_1_1D", "lin", "c-lin")

'Approximation: tot'

,$SR$
$SR-PDG$,-226
$SR-Deng$,-313
$SR-\Gamma$,-262


8.059492676870482

PDG  $SR-PDG$  Deng $SR-Deng$  $\Gamma$  \
In              Out                                                        
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan    -0.011   nan    -0.011     0.341   
                $\chi_{c0}(1P)$  188     -52.3   272     -75.8       260   
                $\chi_{c1}(1P)$ 67.7     -24.4   138     -49.7       111   
                $\chi_{c2}(1P)$  nan      1.66   7.1       2.6      4.53   
                $\eta_{c}(2S)$   nan -0.000348   nan -0.000348  7.92e-05   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan      -121    20      -134        18   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan     -31.9   7.9     -56.2      4.48   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan      2.17  0.36      1.27     0.615   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan   -0.0116  0.27   -0.0132     0.237   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan    -0.341   3.2    -0.148      7.39   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan   -0.0767   1.5   -0.0391      2.94   

                                $SR-\Gamma$  
In              Out                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$       -0.011  
                $\chi_{c0}(1P)$       -72.4  
                $\chi_{c1}(1P)$       -39.9  
                $\chi_{c2}(1P)$        1.66  
                $\eta_{c}(2S)$    -0.000348  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$         -121  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$        -31.9  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$         2.17  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$      -0.0116  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$       -0.341  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$      -0.0767

'Approximation: E1'

...

PDG  $SR-PDG$  Deng $SR-Deng$  E1-$\Gamma$  \
In              Out                                                           
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan    -0.011   nan    -0.011        0.341   
                $\chi_{c0}(1P)$  188     -52.3   272     -75.8          291   
                $\chi_{c1}(1P)$ 67.7     -22.7   138     -46.3          132   
                $\chi_{c2}(1P)$  nan      3.44   7.1      4.09         5.96   
                $\eta_{c}(2S)$   nan -0.000348   nan -0.000348     7.92e-05   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan      -118    20      -134         17.5   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan     -30.8   7.9     -57.6         4.22   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan      1.54  0.36      1.11        0.503   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan   -0.0101  0.27   -0.0132        0.208   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan    -0.296   3.2    -0.171         5.55   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan    0.0302   1.5     0.116         0.39   

                                $SR-\Gamma$  
In              Out                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$       -0.011  
                $\chi_{c0}(1P)$       -81.1  
                $\chi_{c1}(1P)$       -44.3  
                $\chi_{c2}(1P)$        3.44  
                $\eta_{c}(2S)$    -0.000348  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$         -118  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$        -30.8  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$         1.54  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$      -0.0101  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$       -0.296  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$       0.0302

'Approximation: ELW'

,$SR$
$SR-PDG$,-221
$SR-Deng$,-309
$SR-\Gamma$,-282


11.41197681048496

PDG $SR-PDG$  Deng $SR-Deng$  ELW-$\Gamma$  \
In              Out                                                           
$\psi_{1}(1D)$  $\chi_{c0}(1P)$  188    -52.3   272     -75.8           321   
                $\chi_{c1}(1P)$ 67.7    -22.7   138     -46.3           140   
                $\chi_{c2}(1P)$  nan      3.6   7.1      4.09          6.24   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan     -119    20      -134          17.7   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan    -31.3   7.9     -57.6          4.29   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan     1.62  0.36      1.11         0.527   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan   -0.452  0.27   -0.0132          9.27   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan  -0.0184   3.2    -0.171         0.345   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan  0.00708   1.5     0.116        0.0916   

                                $SR-\Gamma$  
In              Out                          
$\psi_{1}(1D)$  $\chi_{c0}(1P)$       -89.5  
                $\chi_{c1}(1P)$         -47  
                $\chi_{c2}(1P)$         3.6  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$         -119  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$        -31.3  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$         1.62  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$       -0.452  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$      -0.0184  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$      0.00708

## Sumrules for the article

In [33]:
def getWidths(pname, prefix, cfgname):
    my_full = getRs(pname, prefix, cfgname, suffix="")
    my_full.rename(columns={"$\Gamma$": "FULL", "$r^{(0)}$": "r0", "$r^{(2)}$": "r2"}, inplace=True)
    my_lwa = getRs(pname, prefix, cfgname, suffix="ELW")
    my_lwa.rename(columns={"$\Gamma$": "LWA", "$r^{(0)}$": "r0lwa", "$r^{(2)}$": "r2lwa"}, inplace=True)
    deng = pd.concat([getDengTrans(prefix, cfg) for\
                          cfg in data_pack[cfgname.split("-")[0]]
                     ], sort=False, axis=0)
    pdg, pdg_err = getPDGTrans(cfgname.split("-")[0])
    pdg = pdg.reindex(my_full.index)
    pdg_err = pdg_err.reindex(my_full.index)
    deng = deng.reindex(my_full.index.values)
    res = pd.concat([pdg, my_full, my_lwa, deng], axis=1)
    err = res*sp.nan
    err["PDG"] = pdg_err
    return res, err.abs()


In [34]:
def getSumrule(pname, prefix, cfgname):
    pretty_pname = "${}$".format(code2name(pname))
    
    widths, widths_err = getWidths(pname, prefix, cfgname)
    masses = getMasses(prefix, cfgname)
    pprops = getPprops(prefix, cfgname)
    
    sumrule_prefactors = []
    
    abovethr = widths.query("Out == r'{}'".format(pretty_pname))
    abovethr_pcls = [p[0] for p in abovethr.index.values]
    sumrule_prefactors.append(
        8*sp.pi**2*masses.loc[abovethr_pcls, "E"]**3\
        /(masses.loc[abovethr_pcls, "E"]**2 - masses.loc[pretty_pname, "E"]**2)**3\
        *pprops.loc[abovethr_pcls, "xJ"]
    )
    sumrule_prefactors[0].index = pd.Index([(v, pretty_pname) for v in sumrule_prefactors[0].index.values])
    
    belowthr = widths.query("In == r'{}'".format(pretty_pname))
    belowthr_pcls = [p[1] for p in belowthr.index.values]
    sumrule_prefactors.append(
        8*sp.pi**2*masses.loc[pretty_pname, "E"]**3\
        /(masses.loc[pretty_pname, "E"]**2 - masses.loc[belowthr_pcls, "E"]**2)**3\
        *pprops.loc[pretty_pname, "xJ"]
    )
    sumrule_prefactors[1].index = pd.Index([(pretty_pname, v) for v in sumrule_prefactors[1].index.values])
    
    sumrule_prefactors_cnct = pd.concat(sumrule_prefactors, axis=0)
    sumrule_prefactors_cnct = sumrule_prefactors_cnct.reindex(widths.index.values)
    
    sumrule_prefactors_cnct = sumrule_prefactors_cnct*(widths.loc[:, "r2"] - widths.loc[:, "r0"])\
                *(0.19732697)**2*10**4*10**3
    sumrule = widths.drop(columns=["r0", "r2", "r0lwa", "r2lwa"])\
                .apply(\
                        lambda s: s*sumrule_prefactors_cnct.values\
                    )
    sumrule_err = widths_err.drop(columns=["r0", "r2", "r0lwa", "r2lwa"])\
                .apply(\
                        lambda s: s*sumrule_prefactors_cnct.values\
                    )
    
    widths.drop(columns=["r2", "r0", "r0lwa", "r2lwa"], inplace=True)
   
    widths = dfsort(widths, lambda a,b: cmpMultyStatesByCode(prefix, cfgname
                                                         ,[name2code(i[1:-1]) for i in a]
                                                         ,[name2code(i[1:-1]) for i in b]))
    sumrule = sumrule.reindex(widths.index.values)
    sumrule_err = sumrule_err.reindex(widths.index.values) 
    
    def mid(df):
        return (df.max(axis=1) + df.min(axis=1))/2
    def spread(df):
        return (df.max(axis=1) - df.min(axis=1)).abs()/2
    
    sumrule_err = pd.concat([sumrule_err.loc[:, ["PDG", "FULL", "LWA", "Deng"]]
                         ,spread(sumrule.drop(columns=["PDG", "FULL", "LWA"])).rename("E1av")], axis=1)
    sumrule = pd.concat([sumrule.loc[:, ["PDG", "FULL", "LWA", "Deng"]]
                         ,mid(sumrule.drop(columns=["PDG", "FULL", "LWA"])).rename("E1av")], axis=1)
    widths_err = pd.concat([widths_err.loc[:, ["PDG", "FULL", "LWA", "Deng"]]
                         ,spread(widths.drop(columns=["PDG", "FULL", "LWA"])).rename("E1av")], axis=1)
    widths = pd.concat([widths.loc[:, ["PDG", "FULL", "LWA", "Deng"]]
                         ,mid(widths.drop(columns=["PDG", "FULL", "LWA"])).rename("E1av")], axis=1)
    
    masses = dfsort(\
                      masses.loc[belowthr_pcls+abovethr_pcls, :]
                    , lambda a, b: \
                        cmpStatesByName(prefix, cfgname, a[1:-1], b[1:-1]) \
                   ).rename(columns={"E": "M"})
    
    return masses, sumrule, sumrule_err.abs(), widths, widths_err

In [47]:
def sumruleReport(pname, prefix, cfgname):
    pretty_pname = "${}$".format(code2name(pname))
    def numfmt(num):
        return "{:.3g}".format(num)
    
    def fillsubtot(row):
        for source in row.index.values:
            if sp.isnan(row[source]):
                if sp.isnan(row["PDG"]):
                    if sp.isnan(row["FULL"]):
                        row[source] = 0
                    else:
                        row[source] = row["FULL"]
                else:
                    row[source] = row["PDG"]
        return row
    
    masses, sumrule, sumrule_err, widths, widths_err = getSumrule(pname, prefix, cfgname)
    abovethr = [pair[0] for pair in widths.query("Out == r'{}'".format(pretty_pname)).index.values]
    
    tab = LongTable("|c|c|c|c|c|")

    # Header
    tab.add_hline()
    tab.add_row([  "X"
                 , NoEscape(r"$m_X$ [$MeV$]")\
                 , NoEscape(r"$\Gamma({} \rightarrow \gamma X )$".format(code2name(pname)))\
                 , NoEscape(r"$\Gamma(X \rightarrow \gamma {})$".format(code2name(pname)))\
                 , NoEscape(r"$SR_1$ [$\mu b$]")
                ])
    tab.add_hline()

    curN = "1"
    tot = sumrule.iloc[0].fillna(0)*0
    tot_err = sumrule_err.iloc[0].fillna(0)*0
    getN = re.compile("\((\d+)[A-Z]\)\$$")
    for label in masses.index.values:
        if label in abovethr:
            label_pair = (label, pretty_pname)
        else:
            label_pair = (pretty_pname, label)

        # Manage subtotals
        newN = getN.search(label).groups()[0]
        if newN != curN:
            tab.add_hline()
            first = True
            for source in tot.index.values:
                if tot[source] == 0:
                    continue
                sr_err = ""
                if tot_err[source] != 0:
                    sr_err = "$\pm$"+numfmt(sp.sqrt(tot_err[source]))
                if first:
                    tab.add_row([NoEscape(i) for i in ["Subtotal"] + [""]*3 + [numfmt(tot[source])+sr_err+" [{}]".format(source)]])
                else:
                    tab.add_row([NoEscape(i) for i in [""]*4 + [numfmt(tot[source])+sr_err+" [{}]".format(source)]])
                if first:
                    first = False
            tab.add_hline()
            tab.add_hline()
            tot = fillsubtot(sumrule.loc[label_pair].copy())
            tot_err = fillsubtot(sumrule_err.loc[label_pair].copy()**2)
            curN = newN
        else:
            tot += fillsubtot(sumrule.loc[label_pair].copy())
            tot_err += fillsubtot(sumrule_err.loc[label_pair].copy()**2)
            
        # Regular entry
        first=True
        curdata = widths.loc[label_pair, :]
        for source in curdata.index.values:
            sr_err = ""
            if not sp.isnan(sumrule_err.loc[label_pair, source]):
                sr_err = "$\pm$"+numfmt(sumrule_err.loc[label_pair, source])
            srentry =  sumrule.loc[label_pair][source]
            if sp.isnan(srentry) or srentry == 0:
                continue
            
            dw_err = ""
            if not sp.isnan(widths_err.loc[label_pair, source]):
                dw_err = "$\pm$"+numfmt(widths_err.loc[label_pair, source])
            dw = numfmt(widths.loc[label_pair, source])+dw_err+" [{}]".format(source)
            dwentry = ("", dw) if label in abovethr else (dw, "")
            
            if first:
                tab.add_row([NoEscape(i) for i in [label, numfmt(masses.loc[label, "M"]), *dwentry, numfmt(srentry)+sr_err]])
            else:
                tab.add_row([NoEscape(i) for i in ["", "", *dwentry, numfmt(srentry)+sr_err]])
            if first:
                first = False
        tab.add_hline()

    # Final subtotal
    tab.add_hline()
    first = True
    for source in tot.index.values:
        if tot[source] == 0:
            continue
        sr_err = ""
        if tot_err[source] != 0:
            sr_err = "$\pm$"+numfmt(sp.sqrt(tot_err[source]))
        if first:
            tab.add_row([NoEscape(i) for i in ["Subtotal"] + [""]*3 + [numfmt(tot[source])+sr_err+" [{}]".format(source)]])
        else:
            tab.add_row([NoEscape(i) for i in [""]*4 + [numfmt(tot[source])+sr_err+" [{}]".format(source)]])
        if first:
            first = False
    tab.add_hline()
    tab.add_hline()

    # Total
    tot = sumrule.sum(axis=0)
    tot_err = sumrule_err.apply(sp.square).sum(axis=0).apply(sp.sqrt)
    first = True
    for source in tot.index.values:
        if tot[source] == 0:
            continue
        sr_err = ""
        if tot_err[source] != 0:
            sr_err = "$\pm$"+numfmt(tot_err[source])        
        if first:
            tab.add_row([NoEscape(i) for i in ["Total"] + [""]*3 + [numfmt(tot[source])+sr_err+" [{}]".format(source)]])
        else:
            tab.add_row([NoEscape(i) for i in [""]*4 + [numfmt(tot[source])+sr_err+" [{}]".format(source)]])
        if first:
            first = False
    tab.add_hline()
    
    return tab

In [48]:
def doReport(pname, prefix, cfgname):
    masses, sr, sr_err, widths, widths_err = getSumrule(pname, prefix, cfgname)
    display(sr.sum(axis=0))
    display(widths)
    display(sr)
    sumruleReport(pname, prefix, cfgname).generate_tex("export/sr-article.{}.{}".format(pname, cfgname))

### Bottomonium sumrules

In [49]:
doReport("yps_1S", "scr", "b-scr")

PDG         0
FULL   -0.648
LWA    -0.558
Deng    -1.52
E1av     -1.2
dtype: float64

,,PDG,FULL,LWA,Deng,E1av
In,Out,,,,,
$\Upsilon(1S)$,$\eta_{b}(1S)$,nan,0.0107,nan,0.01,0.0105
$\chi_{b0}(1P)$,$\Upsilon(1S)$,nan,24.2,26.9,27.5,26.9
$\chi_{b1}(1P)$,$\Upsilon(1S)$,nan,30.2,32.5,31.9,33
$\chi_{b2}(1P)$,$\Upsilon(1S)$,nan,36.1,35.7,31.8,36
$\chi_{b0}(2P)$,$\Upsilon(1S)$,nan,4.44,3.97,5.54,6.95
$\chi_{b1}(2P)$,$\Upsilon(1S)$,nan,10.7,10,10.8,8.95
$\chi_{b2}(2P)$,$\Upsilon(1S)$,nan,17,14.1,12.5,10.5
$\chi_{b0}(3P)$,$\Upsilon(1S)$,nan,1.37,1.01,1.87,3.35
$\chi_{b1}(3P)$,$\Upsilon(1S)$,nan,5.56,4.83,6.41,4.05


,,PDG,FULL,LWA,Deng,E1av
In,Out,,,,,
$\Upsilon(1S)$,$\eta_{b}(1S)$,NaN,-0.542,NaN,-0.505,-0.531
$\chi_{b0}(1P)$,$\Upsilon(1S)$,NaN,-1.55,-1.73,-1.77,-1.73
$\chi_{b1}(1P)$,$\Upsilon(1S)$,NaN,-2.4,-2.59,-2.54,-2.63
$\chi_{b2}(1P)$,$\Upsilon(1S)$,NaN,3.7,3.66,3.26,3.69
$\chi_{b0}(2P)$,$\Upsilon(1S)$,NaN,-0.0416,-0.0372,-0.0519,-0.0651
$\chi_{b1}(2P)$,$\Upsilon(1S)$,NaN,-0.15,-0.14,-0.151,-0.125
$\chi_{b2}(2P)$,$\Upsilon(1S)$,NaN,0.3,0.248,0.221,0.186
$\chi_{b0}(3P)$,$\Upsilon(1S)$,NaN,-0.00561,-0.00414,-0.00763,-0.0137
$\chi_{b1}(3P)$,$\Upsilon(1S)$,NaN,-0.0355,-0.0308,-0.0409,-0.0259


In [50]:
doReport("yps_2S", "scr", "b-scr")

PDG     0.963
FULL   -0.121
LWA    -0.554
Deng   -0.924
E1av    -1.02
dtype: float64

PDG   FULL  LWA  Deng   E1av
In              Out                                            
$\Upsilon(2S)$  $\eta_{b}(1S)$  0.0125 0.0737  nan 0.066 0.0435
                $\chi_{b0}(1P)$   1.22   1.19 1.16  1.09   1.29
                $\chi_{b1}(1P)$   2.21   2.28 2.26  2.17   2.08
                $\chi_{b2}(1P)$   2.29   2.58 2.62  2.62   2.25
$\chi_{b0}(2P)$ $\Upsilon(2S)$     nan   13.2 14.2  14.4   12.7
$\chi_{b1}(2P)$ $\Upsilon(2S)$     nan   15.3 16.2  15.3   14.6
$\chi_{b2}(2P)$ $\Upsilon(2S)$     nan   16.7 16.9  15.3   16.5
$\chi_{b0}(3P)$ $\Upsilon(2S)$     nan   2.18 1.62  2.55   3.25
$\chi_{b1}(3P)$ $\Upsilon(2S)$     nan   5.04 4.23  5.63   4.37
$\chi_{b2}(3P)$ $\Upsilon(2S)$     nan   7.51 5.98  6.72    5.7

PDG     FULL     LWA     Deng     E1av
In              Out                                                        
$\Upsilon(2S)$  $\eta_{b}(1S)$  -0.00065 -0.00384     NaN -0.00344 -0.00227
                $\chi_{b0}(1P)$    -3.27    -3.19   -3.12    -2.93    -3.47
                $\chi_{b1}(1P)$    -5.76    -5.94    -5.9    -5.67    -5.44
                $\chi_{b2}(1P)$     9.99     11.3    11.4     11.4     9.83
$\chi_{b0}(2P)$ $\Upsilon(2S)$       NaN    -5.72   -6.12    -6.23    -5.47
$\chi_{b1}(2P)$ $\Upsilon(2S)$       NaN    -7.45   -7.89    -7.47    -7.12
$\chi_{b2}(2P)$ $\Upsilon(2S)$       NaN     10.7    10.9     9.85     10.6
$\chi_{b0}(3P)$ $\Upsilon(2S)$       NaN  -0.0872 -0.0648   -0.102    -0.13
$\chi_{b1}(3P)$ $\Upsilon(2S)$       NaN    -0.28  -0.236   -0.313   -0.243
$\chi_{b2}(3P)$ $\Upsilon(2S)$       NaN    0.548   0.437    0.491    0.416

In [51]:
doReport("yps_3S", "scr", "b-scr")

PDG      1.81
FULL   -0.502
LWA    -0.745
Deng    -2.47
E1av    -2.85
dtype: float64

PDG     FULL    LWA   Deng    E1av
In              Out                                                  
$\Upsilon(3S)$  $\eta_{b}(1S)$  0.0104   0.0795    nan  0.071   0.041
                $\chi_{b0}(1P)$ 0.0549    0.116 0.0436  0.097    0.08
                $\chi_{b1}(1P)$ 0.0183 9.28e-05 0.0413 0.0005  0.0853
                $\chi_{b2}(1P)$  0.201    0.153  0.455   0.14   0.671
                $\eta_{b}(2S)$     nan   0.0122    nan  0.011 0.00559
                $\chi_{b0}(2P)$    1.2     1.31    1.3   1.21    1.43
                $\chi_{b1}(2P)$   2.56     2.66   2.65   2.61    2.44
                $\chi_{b2}(2P)$   2.66     3.18   3.23   3.16    2.76
$\chi_{b0}(3P)$ $\Upsilon(3S)$     nan     7.59   7.98   7.95     7.7
$\chi_{b1}(3P)$ $\Upsilon(3S)$     nan     9.38   9.84   10.3    10.2
$\chi_{b2}(3P)$ $\Upsilon(3S)$     nan     11.2   11.5   10.8    12.4

PDG      FULL      LWA      Deng  \
In              Out                                                      
$\Upsilon(3S)$  $\eta_{b}(1S)$  -0.000157  -0.00121      NaN  -0.00108   
                $\chi_{b0}(1P)$  -0.00558   -0.0118 -0.00443  -0.00987   
                $\chi_{b1}(1P)$  -0.00109 -5.53e-06 -0.00246 -2.98e-05   
                $\chi_{b2}(1P)$    0.0154    0.0117   0.0348    0.0107   
                $\eta_{b}(2S)$        NaN  -0.00327      NaN  -0.00296   
                $\chi_{b0}(2P)$     -7.62     -8.35    -8.25     -7.69   
                $\chi_{b1}(2P)$     -14.9     -15.5    -15.5     -15.2   
                $\chi_{b2}(2P)$      24.4      29.1     29.6      28.9   
$\chi_{b0}(3P)$ $\Upsilon(3S)$        NaN     -11.8    -12.4     -12.3   
$\chi_{b1}(3P)$ $\Upsilon(3S)$        NaN     -14.6    -15.3       -16   
$\chi_{b2}(3P)$ $\Upsilon(3S)$        NaN      20.6     21.1      19.9   

                                     E1av  
In              Out                        
$\Upsilon(3S)$  $\eta_{b}(1S)$  -0.000623  
                $\chi_{b0}(1P)$  -0.00814  
                $\chi_{b1}(1P)$  -0.00508  
                $\chi_{b2}(1P)$    0.0513  
                $\eta_{b}(2S)$    -0.0015  
                $\chi_{b0}(2P)$     -9.09  
                $\chi_{b1}(2P)$     -14.2  
                $\chi_{b2}(2P)$      25.3  
$\chi_{b0}(3P)$ $\Upsilon(3S)$      -11.9  
$\chi_{b1}(3P)$ $\Upsilon(3S)$      -15.9  
$\chi_{b2}(3P)$ $\Upsilon(3S)$       22.9

###  Charmonium Screened Sumrules

In [52]:
doReport("psi_1S", "scr", "c-scr")

PDG    -32.1
FULL   -29.4
LWA    -22.4
Deng   -54.8
E1av   -35.8
dtype: float64

,,PDG,FULL,LWA,Deng,E1av
In,Out,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,2.79,nan,2.44,1.98
$\chi_{c0}(1P)$,$\psi(1S)$,151,142,183,179,146
$\chi_{c1}(1P)$,$\psi(1S)$,288,310,382,319,296
$\chi_{c2}(1P)$,$\psi(1S)$,374,510,490,292,378
$\eta_{c}(2S)$,$\psi(1S)$,nan,2.91,nan,2.29,4.71
$\chi_{c0}(2P)$,$\psi(1S)$,nan,1.89,3.3,2.3,37.6
$\chi_{c1}(2P)$,$\psi(1S)$,nan,74.4,51.9,88,51
$\chi_{c2}(2P)$,$\psi(1S)$,nan,237,132,93,86


,,PDG,FULL,LWA,Deng,E1av
In,Out,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,-13.3,-23.6,NaN,-20.6,-16.7
$\chi_{c0}(1P)$,$\psi(1S)$,-20.9,-19.6,-25.3,-24.7,-20.2
$\chi_{c1}(1P)$,$\psi(1S)$,-32,-34.5,-42.5,-35.4,-32.9
$\chi_{c2}(1P)$,$\psi(1S)$,34.1,46.5,44.6,26.6,34.5
$\eta_{c}(2S)$,$\psi(1S)$,NaN,-0.0893,NaN,-0.0702,-0.145
$\chi_{c0}(2P)$,$\psi(1S)$,NaN,-0.0222,-0.0387,-0.027,-0.442
$\chi_{c1}(2P)$,$\psi(1S)$,NaN,-1.58,-1.1,-1.86,-1.08
$\chi_{c2}(2P)$,$\psi(1S)$,NaN,3.36,1.86,1.32,1.22


In [53]:
doReport("psi_2S", "scr", "c-scr")

PDG       13
FULL   -29.4
LWA    -19.1
Deng   -83.8
E1av   -56.5
dtype: float64

PDG  FULL    LWA  Deng  E1av
In              Out                                           
$\psi(2S)$      $\eta_{c}(1S)$      1  10.6    nan   7.8  5.27
                $\chi_{c0}(1P)$  28.8  36.6   33.5    22    48
                $\chi_{c1}(1P)$  28.7    56   54.7    45  42.5
                $\chi_{c2}(1P)$    28  44.5   47.5    46  32.1
                $\eta_{c}(2S)$  0.206 0.233    nan  0.19 0.127
$\chi_{c0}(2P)$ $\psi(2S)$        nan   108    128    99  89.5
$\chi_{c1}(2P)$ $\psi(2S)$        nan   102    116   155   122
$\chi_{c2}(2P)$ $\psi(2S)$        nan   200    207   150   202
$\chi_{c0}(3P)$ $\psi(2S)$        nan  6.62 0.0396   9.1    16
$\chi_{c1}(3P)$ $\psi(2S)$        nan  68.1   40.1    74  51.5
$\chi_{c2}(3P)$ $\psi(2S)$        nan   136   75.1    76    73

PDG   FULL      LWA   Deng   E1av
In              Out                                                  
$\psi(2S)$      $\eta_{c}(1S)$  -0.0449 -0.479      NaN -0.351 -0.237
                $\chi_{c0}(1P)$   -18.6  -23.7    -21.6  -14.2    -31
                $\chi_{c1}(1P)$     -31  -60.6    -59.2  -48.7  -45.9
                $\chi_{c2}(1P)$    83.8    133      142    138   96.2
                $\eta_{c}(2S)$    -21.2    -24      NaN  -19.6    -13
$\chi_{c0}(2P)$ $\psi(2S)$          NaN  -81.4    -96.7  -74.9  -67.7
$\chi_{c1}(2P)$ $\psi(2S)$          NaN   -105     -120   -160   -125
$\chi_{c2}(2P)$ $\psi(2S)$          NaN    129      134   96.9    130
$\chi_{c0}(3P)$ $\psi(2S)$          NaN -0.309 -0.00185 -0.425 -0.748
$\chi_{c1}(3P)$ $\psi(2S)$          NaN  -4.27    -2.52  -4.65  -3.23
$\chi_{c2}(3P)$ $\psi(2S)$          NaN   7.86     4.35   4.41   4.23

In [54]:
doReport("psi_1_1D", "scr", "c-scr")

PDG    -76.8
FULL    -299
LWA     -325
Deng    -268
E1av    -449
dtype: float64

PDG    FULL    LWA  Deng  E1av
In              Out                                            
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan   0.386    nan   nan   nan
                $\chi_{c0}(1P)$  188     253    318   261   342
                $\chi_{c1}(1P)$ 67.7     115    148   135   114
                $\chi_{c2}(1P)$  nan    4.73   6.61   8.1   5.7
                $\eta_{c}(2S)$   nan 0.00011    nan   nan   nan
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan    22.5   22.1    12  31.5
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan    5.67   5.42   9.8  14.9
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan   0.812  0.692  0.46  1.13
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan   0.448   9.49  0.39  4.87
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan    4.35  0.392     2  2.21
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan     1.5 0.0791  0.79  0.95

PDG      FULL      LWA    Deng    E1av
In              Out                                                     
$\psi_{1}(1D)$  $\eta_{c}(1S)$    NaN   -0.0124      NaN     NaN     nan
                $\chi_{c0}(1P)$ -52.3     -70.5    -88.5   -72.7   -95.1
                $\chi_{c1}(1P)$ -24.5     -41.6    -53.4   -48.8     -41
                $\chi_{c2}(1P)$   NaN      1.65      2.3    2.82    1.99
                $\eta_{c}(2S)$    NaN -0.000483      NaN     NaN     nan
$\chi_{c0}(2P)$ $\psi_{1}(1D)$    NaN      -151     -149   -80.7    -212
$\chi_{c1}(2P)$ $\psi_{1}(1D)$    NaN     -40.3    -38.6   -69.7    -106
$\chi_{c2}(2P)$ $\psi_{1}(1D)$    NaN      2.87     2.45    1.63    3.99
$\chi_{c0}(3P)$ $\psi_{1}(1D)$    NaN   -0.0394   -0.835 -0.0343  -0.428
$\chi_{c1}(3P)$ $\psi_{1}(1D)$    NaN    -0.363  -0.0327  -0.167  -0.184
$\chi_{c2}(3P)$ $\psi_{1}(1D)$    NaN   -0.0534 -0.00281 -0.0281 -0.0338

### Charmonium Linear

In [55]:
doReport("psi_1S", "lin", "c-lin")

PDG    -31.4
FULL   -26.6
LWA    -20.5
Deng   -51.9
E1av     -35
dtype: float64

,,PDG,FULL,LWA,Deng,E1av
In,Out,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,2.53,nan,2.39,1.98
$\chi_{c0}(1P)$,$\psi(1S)$,151,138,177,172,146
$\chi_{c1}(1P)$,$\psi(1S)$,288,301,367,306,296
$\chi_{c2}(1P)$,$\psi(1S)$,374,490,471,284,378
$\eta_{c}(2S)$,$\psi(1S)$,nan,3.04,nan,2.64,4.71
$\chi_{c0}(2P)$,$\psi(1S)$,nan,2.66,1.66,6.1,37.6
$\chi_{c1}(2P)$,$\psi(1S)$,nan,69.1,46.4,81,51
$\chi_{c2}(2P)$,$\psi(1S)$,nan,231,129,93,86


,,PDG,FULL,LWA,Deng,E1av
In,Out,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,-13.3,-21.3,NaN,-20.2,-16.7
$\chi_{c0}(1P)$,$\psi(1S)$,-20.9,-19.1,-24.4,-23.8,-20.2
$\chi_{c1}(1P)$,$\psi(1S)$,-31.8,-33.3,-40.6,-33.8,-32.7
$\chi_{c2}(1P)$,$\psi(1S)$,34.6,45.3,43.6,26.3,35
$\eta_{c}(2S)$,$\psi(1S)$,NaN,-0.093,NaN,-0.0809,-0.145
$\chi_{c0}(2P)$,$\psi(1S)$,NaN,-0.0313,-0.0195,-0.0716,-0.442
$\chi_{c1}(2P)$,$\psi(1S)$,NaN,-1.45,-0.973,-1.7,-1.07
$\chi_{c2}(2P)$,$\psi(1S)$,NaN,3.37,1.88,1.36,1.25


In [56]:
doReport("psi_2S", "lin", "c-lin")

PDG     12.6
FULL   -25.3
LWA    -14.6
Deng    -114
E1av   -55.8
dtype: float64

PDG  FULL   LWA  Deng  E1av
In              Out                                          
$\psi(2S)$      $\eta_{c}(1S)$      1  9.98   nan  8.08  5.27
                $\chi_{c0}(1P)$  28.8  37.2  34.1    22    48
                $\chi_{c1}(1P)$  28.7  52.2    51    42  42.5
                $\chi_{c2}(1P)$    28  41.8  44.4    38  32.1
                $\eta_{c}(2S)$  0.206 0.211   nan  0.19 0.127
$\chi_{c0}(2P)$ $\psi(2S)$        nan  94.4   111   121  89.5
$\chi_{c1}(2P)$ $\psi(2S)$        nan    91   102   139   122
$\chi_{c2}(2P)$ $\psi(2S)$        nan   177   181   135   202
$\chi_{c0}(3P)$ $\psi(2S)$        nan  10.4 0.466    17    16
$\chi_{c1}(3P)$ $\psi(2S)$        nan  86.6  37.5    94  51.5
$\chi_{c2}(3P)$ $\psi(2S)$        nan   192  86.7    97    73

PDG   FULL     LWA   Deng   E1av
In              Out                                                 
$\psi(2S)$      $\eta_{c}(1S)$  -0.0449 -0.449     NaN -0.363 -0.237
                $\chi_{c0}(1P)$   -18.6    -24     -22  -14.2    -31
                $\chi_{c1}(1P)$   -31.1  -56.6   -55.3  -45.6  -46.1
                $\chi_{c2}(1P)$    83.5    125     132    113   95.8
                $\eta_{c}(2S)$    -21.2  -21.7     NaN  -19.6    -13
$\chi_{c0}(2P)$ $\psi(2S)$          NaN  -71.4   -83.9  -91.5  -67.7
$\chi_{c1}(2P)$ $\psi(2S)$          NaN  -93.7    -105   -143   -125
$\chi_{c2}(2P)$ $\psi(2S)$          NaN    115     118   87.9    131
$\chi_{c0}(3P)$ $\psi(2S)$          NaN -0.302 -0.0135 -0.493 -0.465
$\chi_{c1}(3P)$ $\psi(2S)$          NaN  -3.45   -1.49  -3.74  -2.05
$\chi_{c2}(3P)$ $\psi(2S)$          NaN   6.57    2.96   3.31   2.49

In [57]:
doReport("psi_1_1D", "lin", "c-lin")

PDG    -76.7
FULL    -262
LWA     -286
Deng    -313
E1av    -449
dtype: float64

PDG     FULL    LWA  Deng  E1av
In              Out                                             
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan    0.341    nan   nan   nan
                $\chi_{c0}(1P)$  188      260    321   272   342
                $\chi_{c1}(1P)$ 67.7      111    140   138   114
                $\chi_{c2}(1P)$  nan     4.53   6.24   7.1   5.7
                $\eta_{c}(2S)$   nan 7.92e-05    nan   nan   nan
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan       18   17.7    20  31.5
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan     4.48   4.29   7.9  14.9
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan    0.615  0.527  0.36  1.13
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan    0.237   9.27  0.27  4.87
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan     7.39  0.345   3.2  2.21
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan     2.94 0.0916   1.5  0.95

PDG      FULL      LWA    Deng    E1av
In              Out                                                     
$\psi_{1}(1D)$  $\eta_{c}(1S)$    NaN    -0.011      NaN     NaN     nan
                $\chi_{c0}(1P)$ -52.3     -72.4    -89.5   -75.8   -95.1
                $\chi_{c1}(1P)$ -24.4     -39.9    -50.4   -49.7   -40.9
                $\chi_{c2}(1P)$   NaN      1.66     2.28     2.6    2.08
                $\eta_{c}(2S)$    NaN -0.000348      NaN     NaN     nan
$\chi_{c0}(2P)$ $\psi_{1}(1D)$    NaN      -121     -119    -134    -212
$\chi_{c1}(2P)$ $\psi_{1}(1D)$    NaN     -31.9    -30.6   -56.2    -106
$\chi_{c2}(2P)$ $\psi_{1}(1D)$    NaN      2.17     1.86    1.27    3.98
$\chi_{c0}(3P)$ $\psi_{1}(1D)$    NaN   -0.0116   -0.452 -0.0132  -0.238
$\chi_{c1}(3P)$ $\psi_{1}(1D)$    NaN    -0.341  -0.0159  -0.148  -0.102
$\chi_{c2}(3P)$ $\psi_{1}(1D)$    NaN   -0.0767 -0.00239 -0.0391 -0.0247

In [58]:
raise KeyboardInterrupt("Enough for output")

KeyboardInterrupt: Enough for output